# Team USA Olympic & Paralympic Data Pipeline
### From Raw CSVs → BigQuery-Ready Unified Tables

**Purpose:** Acquire, profile, clean, merge, and enrich Olympic + Paralympic data for Team USA athletes across all available Games years.

**Output Tables:**
- `team_usa_athletes` — One row per unique athlete (identity, physical attributes, classification codes, career stats, AI-generated profiles, embeddings)
- `team_usa_results` — One row per athlete × event × Games (the fact table)

**Pipeline Phases:**
- **Phase 0:** Data acquisition & profiling (assess raw sources, coverage, quality)
- **Phase 1:** Olympic athlete backbone (keithgalli bios + results → career stats)
- **Phase 2:** Paralympic athlete backbone (piterfm athletes + katiepress medalists → classification codes)
- **Phase 3:** Unify athletes table (merge Olympic + Paralympic, normalize, deduplicate)
- **Phase 4:** Unified results table (all medal/event records)
- **Phase 5:** Enrichment (Gemini profile summaries, vector embeddings)
- **Phase 6:** Final validation & GCS export

**Data sources:** gs://class-demo/team-usa/raw/  
**Processed output:** gs://class-demo/team-usa/processed/  
**Enriched output:** gs://class-demo/team-usa/enriched/

Lets get started with:
## Phase 0

## 1. Environment Setup

Our raw data is staged in GCS at `gs://class-demo/team-usa/raw/`. We pull it into the local runtime for pandas processing. Colab Enterprise handles GCS authentication automatically through the attached service account — no keys needed.

In [ ]:
import pandas as pd
import os
import glob
import json
from collections import defaultdict

# Configuration
BUCKET = 'gs://class-demo/team-usa'
RAW_PATH = f'{BUCKET}/raw'
LOCAL_DIR = '/tmp/olympic-data'

os.makedirs(LOCAL_DIR, exist_ok=True)

PROJECT_ID = "qwiklabs-gcp-01-bafc8841fc77"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

print(f'Project: {PROJECT_ID}')
print(f'Region: {REGION}')
print(f'Source: {RAW_PATH}')
print(f'Local: {LOCAL_DIR}')

## 2. Pull Raw Data from GCS

Download all raw CSV files from the staging bucket. The `-m` flag enables parallel transfers. This should take under a minute for the full set.

In [ ]:
!gsutil -m cp -r {RAW_PATH}/* {LOCAL_DIR}/

# Count what we got
csv_count = !find {LOCAL_DIR} -name "*.csv" | wc -l
print(f'\nTotal CSV files downloaded: {csv_count[0].strip()}')

## 3. Inventory All Files

Before diving into profiling, let's see exactly what landed — file names, sizes, and directory structure. This confirms the downloads are complete and gives us a roadmap for the profiling loop.

In [ ]:
# Walk the directory tree and catalog every CSV
file_inventory = []

for root, dirs, files in os.walk(LOCAL_DIR):
    # Skip .git directories
    dirs[:] = [d for d in dirs if d != '.git']
    for f in sorted(files):
        if f.endswith('.csv'):
            full_path = os.path.join(root, f)
            size_mb = os.path.getsize(full_path) / (1024 * 1024)
            rel_path = full_path.replace(LOCAL_DIR + '/', '')
            dataset = rel_path.split('/')[0]
            file_inventory.append({
                'dataset': dataset,
                'file': f,
                'rel_path': rel_path,
                'full_path': full_path,
                'size_mb': size_mb
            })

inv_df = pd.DataFrame(file_inventory)

# Summary by dataset
print('FILES BY DATASET:\n')
for dataset, group in inv_df.groupby('dataset'):
    total_mb = group['size_mb'].sum()
    print(f'{dataset}/ ({len(group)} files, {total_mb:.1f} MB total)')
    for _, row in group.iterrows():
        print(f'  {row["file"]:45s} {row["size_mb"]:8.2f} MB')
    print()

## 4. Profile Every CSV

This is the core profiling loop. For each file we capture:

- **Shape** — row and column counts
- **Schema** — column names, data types, and null percentages
- **Sample data** — first 3 rows to understand format and content
- **Team USA count** — how many rows match USA after filtering (our target population)
- **Event samples** — particularly important for Paralympic files, where classification codes like T54 or S6 may appear inside event names rather than in a dedicated column

The output is intentionally verbose. We need the full picture before making merge decisions.

In [ ]:
# Store profiling results for later summary
profiles = {}

for _, row in inv_df.iterrows():
    csv_file = row['full_path']
    rel_path = row['rel_path']

    try:
        df = pd.read_csv(csv_file, low_memory=False)
    except Exception as e:
        print(f'\nERROR reading {rel_path}: {e}')
        continue

    # Store for later use
    profiles[rel_path] = {
        'rows': df.shape[0],
        'cols': df.shape[1],
        'columns': list(df.columns),
        'dtypes': {col: str(df[col].dtype) for col in df.columns},
        'null_pcts': {col: round(df[col].isnull().mean() * 100, 1) for col in df.columns}
    }

    # ---- Print profile ----
    print(f"\n{'='*70}")
    print(f'FILE: {rel_path}')
    print(f'Shape: {df.shape[0]:,} rows x {df.shape[1]} columns')

    # Schema with null counts
    print(f'\nSchema:')
    for col in df.columns:
        null_count = df[col].isnull().sum()
        null_pct = df[col].isnull().mean() * 100
        print(f'  {col:35s} {str(df[col].dtype):10s} nulls: {null_count:>8,} ({null_pct:5.1f}%)')

    # Sample rows
    print(f'\nFirst 3 rows:')
    print(df.head(3).to_string())

    # ---- Team USA filter ----
    country_cols = [c for c in df.columns if any(
        term in c.lower() for term in ['country', 'noc', 'nationality', 'team']
    )]

    usa_found = False
    for col in country_cols:
        usa_mask = df[col].astype(str).str.contains(
            r'\bUSA\b|United States|United States of America',
            case=False, na=False, regex=True
        )
        usa_count = usa_mask.sum()
        if usa_count > 0:
            usa_found = True
            profiles[rel_path]['usa_count'] = usa_count
            profiles[rel_path]['usa_filter_col'] = col
            print(f'\n  🇺🇸 USA rows (filtered on \'{col}\'): {usa_count:,}')
            print(f'  USA sample:')
            print(df[usa_mask].head(3).to_string())
            break

    if not usa_found:
        print(f'\n  ⚠️  No USA rows found. Country-like columns checked: {country_cols}')

    # ---- Event/discipline samples ----
    event_cols = [c for c in df.columns if any(
        term in c.lower() for term in ['event', 'discipline', 'sport']
    )]
    for col in event_cols:
        unique_vals = df[col].dropna().unique()
        print(f'\n  Sample values from \'{col}\' ({len(unique_vals):,} unique):')
        for val in unique_vals[:15]:
            print(f'    {val}')

    print()

## 5. Summary Dashboard

Now that we've profiled everything, let's consolidate the key findings into a single view. This shows each file's row count, column count, and Team USA coverage at a glance — making it easy to compare sources and spot gaps.

In [ ]:
# Build summary table
summary_rows = []
for path, profile in profiles.items():
    summary_rows.append({
        'File': path,
        'Rows': profile['rows'],
        'Columns': profile['cols'],
        'USA Rows': profile.get('usa_count', 0),
        'USA Filter': profile.get('usa_filter_col', 'N/A'),
        'Key Columns': ', '.join(profile['columns'][:6]) + ('...' if len(profile['columns']) > 6 else '')
    })

summary_df = pd.DataFrame(summary_rows)
summary_df = summary_df.sort_values('USA Rows', ascending=False)

print('DATASET SUMMARY (sorted by USA row count):\n')
print(summary_df.to_string(index=False))

print(f'\n--- TOTALS ---')
print(f'Total files profiled: {len(profiles)}')
print(f'Total rows across all files: {sum(p["rows"] for p in profiles.values()):,}')
print(f'Total USA rows (sum, includes duplicates across datasets): {sum(p.get("usa_count", 0) for p in profiles.values()):,}')

## 6. Critical Questions: Paralympic Classification Codes

This is the single most important unknown. Do the Paralympic datasets include classification codes (T54, S6, F20, etc.) as structured data? Or are they embedded in event names like "Men's 100m T54"? Or missing entirely?

We look at both Paralympic sources and search for classification patterns in all text columns.

In [ ]:
import re

# Classification code pattern: letter(s) followed by numbers, e.g., T54, S6, F20, SB14
classification_pattern = re.compile(r'\b[A-Z]{1,2}\d{1,2}\b')

# Check Paralympic datasets specifically
para_files = [row['full_path'] for _, row in inv_df.iterrows()
              if 'paralympic' in row['dataset'].lower() or 'para' in row['dataset'].lower()]

print('PARALYMPIC CLASSIFICATION CODE SEARCH\n')

for csv_file in para_files:
    rel_path = csv_file.replace(LOCAL_DIR + '/', '')
    df = pd.read_csv(csv_file, low_memory=False)

    print(f'\n--- {rel_path} ---')
    print(f'Columns: {list(df.columns)}')

    # Check if any column name suggests classification
    class_cols = [c for c in df.columns if any(
        term in c.lower() for term in ['class', 'category', 'impairment', 'disability']
    )]
    if class_cols:
        print(f'  ✅ Possible classification columns: {class_cols}')
        for col in class_cols:
            print(f'     Sample values: {df[col].dropna().unique()[:20]}')
    else:
        print(f'  ❌ No dedicated classification column found')

    # Search all text columns for classification patterns
    print(f'\n  Searching all text columns for classification patterns (e.g., T54, S6):')
    for col in df.select_dtypes(include='object').columns:
        matches = df[col].fillna('').astype(str).apply(
            lambda x: classification_pattern.findall(x)
        )
        flat_matches = [m for sublist in matches for m in sublist]
        if flat_matches:
            unique_codes = sorted(set(flat_matches))
            print(f'    \'{col}\': Found {len(unique_codes)} unique codes')
            print(f'      Examples: {unique_codes[:25]}')
            # Show sample rows with codes
            has_code = matches.apply(len) > 0
            print(f'      Sample rows:')
            print(df.loc[has_code].head(3)[[col]].to_string())

## 7. Critical Questions: Physical Attributes

The "find athletes with my build" similarity search requires height, weight, and ideally age. Which datasets have these fields, and how well-populated are they — especially for Team USA athletes in the modern era?

In [ ]:
print('PHYSICAL ATTRIBUTE COVERAGE\n')

physical_terms = ['height', 'weight', 'age', 'birth', 'born', 'dob', 'bmi']

for _, row in inv_df.iterrows():
    csv_file = row['full_path']
    rel_path = row['rel_path']
    df = pd.read_csv(csv_file, low_memory=False)

    phys_cols = [c for c in df.columns if any(term in c.lower() for term in physical_terms)]

    if phys_cols:
        print(f'\n{rel_path}')

        # Filter to USA if possible
        country_cols = [c for c in df.columns if any(
            term in c.lower() for term in ['country', 'noc', 'nationality', 'team']
        )]
        usa_df = df
        for col in country_cols:
            mask = df[col].astype(str).str.contains(
                r'\bUSA\b|United States', case=False, na=False, regex=True
            )
            if mask.sum() > 0:
                usa_df = df[mask]
                print(f'  (Filtered to {usa_df.shape[0]:,} USA rows)')
                break

        for col in phys_cols:
            non_null = usa_df[col].notna().sum()
            total = len(usa_df)
            pct = (non_null / total * 100) if total > 0 else 0
            print(f'  {col:25s} {non_null:>6,} / {total:>6,} populated ({pct:.1f}%)')

            # Show sample values
            sample_vals = usa_df[col].dropna().head(5).tolist()
            print(f'    Sample values: {sample_vals}')

## 8. Critical Questions: Winter Paralympics

We want both Summer and Winter. The Olympic datasets clearly cover both. But do the Paralympic datasets include Winter Games (PyeongChang 2018, Beijing 2022, Sochi 2014, etc.)? This determines whether we need additional sources for Winter Paralympic coverage.

In [ ]:
print('WINTER PARALYMPIC COVERAGE SEARCH\n')

winter_terms = ['winter', 'beijing 2022', 'pyeongchang', 'sochi', 'vancouver 2010',
                'turin', 'torino', 'salt lake', 'nagano', 'lillehammer']

for csv_file in para_files:
    rel_path = csv_file.replace(LOCAL_DIR + '/', '')
    df = pd.read_csv(csv_file, low_memory=False)

    print(f'\n--- {rel_path} ---')

    # Check all text columns for winter indicators
    for col in df.select_dtypes(include='object').columns:
        col_text = df[col].fillna('').astype(str).str.lower()
        for term in winter_terms:
            matches = col_text.str.contains(term, na=False)
            if matches.sum() > 0:
                print(f'  ✅ Found \'{term}\' in \'{col}\': {matches.sum()} rows')
                print(f'     Sample: {df.loc[matches, col].unique()[:5]}')

    # Check year/season columns
    year_cols = [c for c in df.columns if any(
        term in c.lower() for term in ['year', 'season', 'games', 'edition']
    )]
    for col in year_cols:
        print(f'  \'{col}\' unique values: {sorted(df[col].dropna().unique())}')

## 9. Schema Comparison Across Datasets

Before we can merge anything, we need to understand how column names and data representations differ across sources. This comparison identifies which fields can be directly joined and which need mapping or renaming.

In [ ]:
print('COLUMN NAMES ACROSS ALL DATASETS\n')

# Group files by dataset
datasets = inv_df.groupby('dataset')['full_path'].apply(list).to_dict()

# For each dataset, get the union of all column names from its files
dataset_columns = {}
for dataset_name, file_list in datasets.items():
    all_cols = set()
    for f in file_list:
        try:
            df = pd.read_csv(f, nrows=0)  # Just read headers
            all_cols.update(df.columns)
        except:
            pass
    dataset_columns[dataset_name] = sorted(all_cols)

for dataset_name, cols in dataset_columns.items():
    print(f'\n{dataset_name}/ ({len(cols)} unique columns):')
    for col in cols:
        print(f'  {col}')

# Find common columns across all datasets
print(f'\n{"="*70}')
print('COLUMN OVERLAP ANALYSIS:')
all_col_sets = list(dataset_columns.values())
if all_col_sets:
    common = set(all_col_sets[0])
    for s in all_col_sets[1:]:
        common &= set(s)
    print(f'\nColumns present in ALL datasets: {sorted(common) if common else "None"}')

    # Columns by frequency
    col_freq = defaultdict(int)
    for cols in all_col_sets:
        for c in cols:
            col_freq[c] += 1

    print(f'\nColumns by frequency (how many datasets contain them):')
    for col, freq in sorted(col_freq.items(), key=lambda x: -x[1]):
        datasets_with = [name for name, cols in dataset_columns.items() if col in cols]
        joined = ', '.join(datasets_with)
        print(f'  {col:35s} {freq}/{len(dataset_columns)}  ({joined})')

## 10. Overlap and Duplication Check

Several of these datasets cover the same time periods (e.g., KeithGalli and heesoo37 both cover pre-2016 Olympics). Before merging, we need to understand where coverage overlaps and which source is more complete in the overlap zones. This prevents double-counting athletes.

In [ ]:
print('TEMPORAL COVERAGE BY DATASET\n')

for _, row in inv_df.iterrows():
    csv_file = row['full_path']
    rel_path = row['rel_path']
    df = pd.read_csv(csv_file, low_memory=False)

    # Look for year-like columns
    year_cols = [c for c in df.columns if any(
        term in c.lower() for term in ['year', 'games', 'edition', 'date']
    )]

    if year_cols:
        print(f'\n{rel_path}')
        for col in year_cols:
            vals = df[col].dropna().unique()
            # Try to extract years
            years = set()
            for v in vals:
                found = re.findall(r'\b(19\d{2}|20\d{2})\b', str(v))
                years.update(found)
            if years:
                year_range = sorted(years)
                print(f'  \'{col}\': {year_range[0]} — {year_range[-1]} ({len(year_range)} distinct years)')
                print(f'    Years: {year_range}')
            else:
                print(f'  \'{col}\': {sorted(vals)[:10]} (could not extract years)')

## 11. Findings Summary

Compile the key findings that will drive the merge strategy. This cell generates a structured report we can bring back to the lab planning conversation.

**Key questions this must answer:**
1. Do we have structured Paralympic classification codes?
2. Which datasets have physical attributes (height/weight) for Team USA?
3. What's the total Team USA record count (Olympic + Paralympic, Summer + Winter)?
4. Do we have Winter Paralympic coverage?
5. What time period does our data span?
6. What's missing that we expected to have?

In [ ]:
print('='*70)
print('FINDINGS SUMMARY')
print('='*70)

print('\n1. DATASET INVENTORY')
print(f'   Total CSV files: {len(profiles)}')
print(f'   Total rows (all countries): {sum(p["rows"] for p in profiles.values()):,}')
print(f'   Total USA rows (pre-dedup): {sum(p.get("usa_count", 0) for p in profiles.values()):,}')

print('\n2. USA ROWS BY SOURCE')
for path, profile in sorted(profiles.items(), key=lambda x: x[1].get('usa_count', 0), reverse=True):
    usa = profile.get('usa_count', 0)
    if usa > 0:
        print(f'   {path:55s} {usa:>6,} USA rows')

print('\n3. ACTION ITEMS')
print('   Review the output above and document:')
print('   [ ] Paralympic classification code status (Section 6)')
print('   [ ] Physical attribute availability (Section 7)')
print('   [ ] Winter Paralympic coverage (Section 8)')
print('   [ ] Preferred source for overlapping time periods (Section 10)')
print('   [ ] Any unexpected gaps or data quality issues')
print('\n   Bring these findings back to plan the unified schema and merge strategy.')

---
## Phase 1: Olympic Athletes

Build the Olympic athlete backbone from keithgalli (bios + results through 2022), then identify any Paris 2024 athletes not already covered.

**Sources:**
- `olympic-keithgalli/bios.csv` → USA athlete identification via NOC
- `olympic-keithgalli/bios_locs.csv` → Structured birth, height, weight
- `olympic-keithgalli/results.csv` → Career stats (Games appearances, medals, sports)
- `olympic-paris2024/athletes.csv` → Gap check for 2024 debut athletes

In [ ]:
# ── Phase 1, Step 1: Load keithgalli backbone ──────────────────────
import warnings
warnings.filterwarnings('ignore')

# bios.csv: filter by NOC to catch USA athletes born abroad
bios = pd.read_csv(os.path.join(LOCAL_DIR, 'olympic-keithgalli', 'bios.csv'), low_memory=False)
usa_bios = bios[bios['NOC'].astype(str).str.contains('United States', case=False, na=False)].copy()

# bios_locs.csv: structured birth/physical data (all countries)
bios_locs = pd.read_csv(os.path.join(LOCAL_DIR, 'olympic-keithgalli', 'bios_locs.csv'), low_memory=False)

# Join USA athletes with their structured data
olympic_backbone = usa_bios[['athlete_id', 'Sex', 'Used name', 'Roles']].merge(
    bios_locs[['athlete_id', 'name', 'born_date', 'born_city', 'born_region',
                'born_country', 'height_cm', 'weight_kg']],
    on='athlete_id', how='left'
)

# Prefer 'Used name' from bios, fall back to bios_locs 'name'
olympic_backbone['display_name'] = olympic_backbone['Used name'].fillna(olympic_backbone['name'])

has_h = olympic_backbone['height_cm'].notna() & (olympic_backbone['height_cm'] > 0)
has_w = olympic_backbone['weight_kg'].notna() & (olympic_backbone['weight_kg'] > 0)

print(f"Keithgalli USA athletes: {len(olympic_backbone):,}")
print(f"  With birth_date:  {olympic_backbone['born_date'].notna().sum():,}")
print(f"  With height (>0): {has_h.sum():,}")
print(f"  With weight (>0): {has_w.sum():,}")
print(f"\nSex distribution:\n{olympic_backbone['Sex'].value_counts().to_string()}")
print(f"\nRoles (sample):\n{olympic_backbone['Roles'].value_counts().head(5).to_string()}")
print(f"\nSample rows:")
olympic_backbone[['athlete_id', 'display_name', 'Sex', 'born_date',
                   'born_city', 'height_cm', 'weight_kg']].head(5)

In [ ]:
# ── Phase 1, Step 2: Career stats from keithgalli results ─────────
results = pd.read_csv(os.path.join(LOCAL_DIR, 'olympic-keithgalli', 'results.csv'), low_memory=False)
usa_results = results[results['noc'].str.upper().str.strip() == 'USA'].copy()

print(f"Results: {len(results):,} total → {len(usa_results):,} USA")
print(f"Unique USA athletes in results: {usa_results['athlete_id'].nunique():,}")
print(f"Year range: {int(usa_results['year'].min())} — {int(usa_results['year'].max())}")
print(f"\n'type' unique values: {sorted(usa_results['type'].dropna().unique())}")
print(f"'medal' unique values: {sorted(usa_results['medal'].dropna().unique())}")
print(f"'discipline' sample values: {sorted(usa_results['discipline'].dropna().unique()[:10])}")

# Aggregate per athlete
def most_common(s):
    mode = s.dropna().mode()
    return mode.iloc[0] if len(mode) > 0 else None

career = usa_results.groupby('athlete_id').agg(
    first_games_year=('year', 'min'),
    last_games_year=('year', 'max'),
    games_count=('year', 'nunique'),
    primary_sport=('discipline', most_common),
    gold_count=('medal', lambda x: (x == 'Gold').sum()),
    silver_count=('medal', lambda x: (x == 'Silver').sum()),
    bronze_count=('medal', lambda x: (x == 'Bronze').sum()),
    games_types=('type', lambda x: ','.join(sorted(x.dropna().unique())))
).reset_index()

career['total_medals'] = career['gold_count'] + career['silver_count'] + career['bronze_count']

print(f"\nCareer stats computed for {len(career):,} athletes")
print(f"  With medals: {(career['total_medals'] > 0).sum():,}")
print(f"  Multi-Games: {(career['games_count'] > 1).sum():,}")
print(f"\n'games_types' unique values (for season derivation):")
for gt in sorted(career['games_types'].unique()):
    count = (career['games_types'] == gt).sum()
    print(f"  {gt:50s} → {count:,} athletes")

In [ ]:
# ── Phase 1, Step 3: Merge backbone with career stats ─────────────
olympic_athletes = olympic_backbone.merge(career, on='athlete_id', how='left')

# Drop the original 'name' from bios_locs before renaming display_name
olympic_athletes = olympic_athletes.drop(columns=['name'], errors='ignore')

# Standardize columns
olympic_athletes = olympic_athletes.rename(columns={
    'Sex': 'gender',
    'display_name': 'name',
    'born_date': 'birth_date',
    'born_city': 'birth_city',
    'born_region': 'birth_region',
    'born_country': 'birth_country'
})
olympic_athletes['games_type'] = 'Olympic'

# Drop intermediate columns
olympic_athletes = olympic_athletes.drop(columns=['Used name', 'Roles'],
                                          errors='ignore')

print(f"Olympic athletes table: {len(olympic_athletes):,} rows")
print(f"  With career results:   {olympic_athletes['games_count'].notna().sum():,}")
print(f"  Bios only (no results): {olympic_athletes['games_count'].isna().sum():,}")

# Verify no duplicate columns
dupes = olympic_athletes.columns[olympic_athletes.columns.duplicated()].tolist()
if dupes:
    print(f"\n⚠️ DUPLICATE COLUMNS: {dupes}")
else:
    print(f"\n✅ No duplicate columns")

print(f"\nColumn completeness:")
for col in ['name', 'gender', 'birth_date', 'birth_city', 'birth_country',
            'height_cm', 'weight_kg', 'primary_sport', 'games_count',
            'first_games_year', 'last_games_year', 'total_medals', 'games_types']:
    if col in olympic_athletes.columns:
        non_null = int(olympic_athletes[col].notna().sum())
        pct = non_null / len(olympic_athletes) * 100
        if col in ['height_cm', 'weight_kg']:
            non_zero = int((olympic_athletes[col].notna() & (olympic_athletes[col] > 0)).sum())
            print(f"  {col:25s} {non_null:>6,} non-null, {non_zero:>6,} non-zero ({non_zero/len(olympic_athletes)*100:.1f}%)")
        else:
            print(f"  {col:25s} {non_null:>6,} / {len(olympic_athletes):,}  ({pct:.1f}%)")

In [ ]:
# ── Phase 1, Step 4: Paris 2024 gap analysis ──────────────────────
paris = pd.read_csv(os.path.join(LOCAL_DIR, 'olympic-paris2024', 'athletes.csv'), low_memory=False)
paris_usa = paris[paris['country_code'] == 'USA'].copy()

# Normalize names: uppercase, sort parts (handles LAST FIRST vs FIRST LAST)
def norm_name(n):
    if pd.isna(n): return ''
    return ' '.join(sorted(str(n).upper().strip().split()))

paris_usa['_norm'] = paris_usa['name'].apply(norm_name)
olympic_athletes['_norm'] = olympic_athletes['name'].apply(norm_name)

matched = paris_usa['_norm'].isin(olympic_athletes['_norm'])

print(f"Paris 2024 USA athletes: {len(paris_usa):,}")
print(f"  Already in keithgalli: {matched.sum():,}")
print(f"  New (not matched):     {(~matched).sum():,}")

if (~matched).sum() > 0:
    print(f"\nSample unmatched (first 15):")
    print(paris_usa[~matched][['name', 'birth_date', 'disciplines']].head(15).to_string())

# Paris 2024 height/weight quality check
h_valid = paris_usa['height'].notna() & (paris_usa['height'] > 0)
w_valid = paris_usa['weight'].notna() & (paris_usa['weight'] > 0)
print(f"\nParis 2024 physical attributes:")
print(f"  Non-zero height: {h_valid.sum():,} / {len(paris_usa):,} ({h_valid.sum()/len(paris_usa)*100:.1f}%)")
print(f"  Non-zero weight: {w_valid.sum():,} / {len(paris_usa):,} ({w_valid.sum()/len(paris_usa)*100:.1f}%)")
if h_valid.sum() > 0:
    print(f"  Height range: {paris_usa.loc[h_valid, 'height'].min():.0f} — {paris_usa.loc[h_valid, 'height'].max():.0f}")
if w_valid.sum() > 0:
    print(f"  Weight range: {paris_usa.loc[w_valid, 'weight'].min():.0f} — {paris_usa.loc[w_valid, 'weight'].max():.0f}")

# Clean up temp column
olympic_athletes = olympic_athletes.drop(columns=['_norm'])

In [ ]:
# ── PHASE 1 QC REPORT ─────────────────────────────────────────────
print('=' * 70)
print('PHASE 1 QC: OLYMPIC ATHLETES')
print('=' * 70)

print(f'\n📊 COUNTS')
print(f'  Total athletes:          {len(olympic_athletes):,}')
has_results = olympic_athletes['games_count'].notna().sum()
print(f'  With career results:     {has_results:,}')
print(f'  Bios only (no results):  {len(olympic_athletes) - has_results:,}')

print(f'\n📅 TEMPORAL RANGE')
print(f'  Earliest Games year: {olympic_athletes["first_games_year"].min()}')
print(f'  Latest Games year:   {olympic_athletes["last_games_year"].max()}')

print(f'\n📏 PHYSICAL ATTRIBUTES')
h = (olympic_athletes['height_cm'].notna() & (olympic_athletes['height_cm'] > 0)).sum()
w = (olympic_athletes['weight_kg'].notna() & (olympic_athletes['weight_kg'] > 0)).sum()
print(f'  Height: {h:,} / {len(olympic_athletes):,} ({h/len(olympic_athletes)*100:.1f}%)')
print(f'  Weight: {w:,} / {len(olympic_athletes):,} ({w/len(olympic_athletes)*100:.1f}%)')

print(f'\n🏅 MEDAL TOTALS (across all years)')
for medal in ['gold_count', 'silver_count', 'bronze_count', 'total_medals']:
    val = olympic_athletes[medal].sum()
    print(f'  {medal:20s} {val:,.0f}')
print(f'  Athletes with 1+ medal: {(olympic_athletes["total_medals"] > 0).sum():,}')

print(f'\n⚧ GENDER SPLIT')
print(olympic_athletes['gender'].value_counts().to_string())

print(f'\n🏟️ GAMES TYPES (season indicators)')
print(olympic_athletes['games_types'].value_counts().head(10).to_string())

print(f'\n🔀 PARIS 2024 GAP')
print(f'  Paris USA athletes:      {len(paris_usa):,}')
print(f'  Already in keithgalli:   {matched.sum():,}')
print(f'  New 2024 athletes:       {(~matched).sum():,}')

print(f'\n📋 SAMPLE ATHLETES (5 random with results):')
sample_cols = ['name', 'gender', 'birth_date', 'height_cm', 'weight_kg',
               'primary_sport', 'games_count', 'total_medals',
               'first_games_year', 'last_games_year', 'games_types']
sample_cols = [c for c in sample_cols if c in olympic_athletes.columns]
print(olympic_athletes.dropna(subset=['games_count'])[sample_cols].sample(5, random_state=42).to_string())

print(f'\n{"=" * 70}')
print('Paste this output back for review before proceeding to Phase 2.')
print('=' * 70)

---
## Phase 2: Paralympic Athletes

Build the Paralympic athlete backbone from three sources:
- `piterfm/2020_Tokyo/athletes.csv` → 199 USA, has dedicated `sport_class` column
- `piterfm/2024_Paris/athletes.csv` → 220 USA, rich biographical fields, classification in event strings
- `katiepress/medal_athlete.csv` → Historical medalists 1960–2018, classification in event strings

**Key task:** Extract and normalize Paralympic classification codes from all three sources.

In [ ]:
# ── Phase 2, Step 1: Tokyo 2020 Paralympic athletes ───────────────
tokyo_para = pd.read_csv(os.path.join(LOCAL_DIR, 'paralympic-piterfm', '2020_Tokyo', 'athletes.csv'), low_memory=False)
tokyo_usa = tokyo_para[tokyo_para['birth_country'].astype(str).str.contains('United States', case=False, na=False) |
                        tokyo_para['country_code'].astype(str).str.upper().eq('USA') |
                        tokyo_para['country'].astype(str).str.contains('United States', case=False, na=False)].copy()

print(f"Tokyo 2020 Paralympic athletes: {len(tokyo_para):,} total → {len(tokyo_usa):,} USA")
print(f"\nColumns: {list(tokyo_usa.columns)}")
print(f"\nsport_class coverage:")
print(f"  Non-null: {tokyo_usa['sport_class'].notna().sum()} / {len(tokyo_usa)}")
print(f"  Unique codes: {tokyo_usa['sport_class'].nunique()}")
print(f"  Sample: {sorted(tokyo_usa['sport_class'].dropna().unique()[:20])}")
print(f"\ndiscipline values: {sorted(tokyo_usa['discipline'].dropna().unique())}")
print(f"\nSample rows:")
tokyo_usa[['name', 'gender', 'birth_date', 'discipline', 'sport_class']].head(10)

In [ ]:
# ── Phase 2, Step 2: Paris 2024 Paralympic athletes ───────────────
paris_para = pd.read_csv(os.path.join(LOCAL_DIR, 'paralympic-piterfm', '2024_Paris', 'athletes.csv'), low_memory=False)
paris_para_usa = paris_para[paris_para['country_code'].astype(str).str.upper().eq('USA')].copy()

print(f"Paris 2024 Paralympic athletes: {len(paris_para):,} total → {len(paris_para_usa):,} USA")

# Check for dedicated classification column
class_cols = [c for c in paris_para_usa.columns if 'class' in c.lower() or 'categ' in c.lower()]
print(f"\nClassification-related columns: {class_cols}")

# Classification lives in 'events' field — extract it
import re

# Known Paralympic classification prefixes
PARA_CLASS_PATTERN = re.compile(
    r'\b(T[1-5]\d|F[1-5]\d|S[1-9]\d?|SB\d{1,2}|SM\d{1,2}|'
    r'B[1-3]|BC[1-4]|C[1-5]|H[1-5]|'
    r'KL[1-3]|VL[1-3]|'
    r'PR[1-3]|PT[S]?[1-5]|'
    r'SH[1-2]|SL[1-4]|SU5|'
    r'W[1-2]|WH[1-2]|'
    r'K4[1-4]|'
    r'J[1-2]|'
    r'M[DS]\d{1,2}|W[DS]\d{1,2}|XD\d{1,2}|'
    r'LW\d{1,2}|LL[1-2]|'
    r'VS[1-2]|PWL)\b'
)

def extract_classifications(text):
    """Extract all unique classification codes from text."""
    if pd.isna(text):
        return None
    codes = set(PARA_CLASS_PATTERN.findall(str(text)))
    return ','.join(sorted(codes)) if codes else None

paris_para_usa['classification_code'] = paris_para_usa['events'].apply(extract_classifications)

cc = paris_para_usa['classification_code']
print(f"\nClassification extraction from 'events':")
print(f"  Extracted: {cc.notna().sum()} / {len(paris_para_usa)} ({cc.notna().sum()/len(paris_para_usa)*100:.1f}%)")
print(f"  Unique code sets: {cc.nunique()}")
print(f"  Sample:")
for _, row in paris_para_usa[cc.notna()][['name', 'events', 'classification_code']].head(5).iterrows():
    print(f"    {row['name']:30s} events={str(row['events'])[:60]:60s} → {row['classification_code']}")

# Bio fields check
bio_fields = ['hobbies', 'occupation', 'education', 'family', 'coach', 'reason',
              'hero', 'influence', 'philosophy', 'sporting_relatives', 'ritual', 'other_sports']
print(f"\nBio field coverage (USA athletes):")
for field in bio_fields:
    if field in paris_para_usa.columns:
        n = paris_para_usa[field].notna().sum()
        print(f"  {field:25s} {n:>4} / {len(paris_para_usa)} ({n/len(paris_para_usa)*100:.1f}%)")

# Physical attributes
h = paris_para_usa['height'].notna() & (paris_para_usa['height'] > 0)
w = paris_para_usa['weight'].notna() & (paris_para_usa['weight'] > 0)
print(f"\nPhysical attributes:")
print(f"  Non-zero height: {h.sum()} / {len(paris_para_usa)} ({h.sum()/len(paris_para_usa)*100:.1f}%)")
print(f"  Non-zero weight: {w.sum()} / {len(paris_para_usa)} ({w.sum()/len(paris_para_usa)*100:.1f}%)")

In [ ]:
# ── Phase 2, Step 3: Historical Paralympic medalists (katiepress) ──
katie = pd.read_csv(os.path.join(LOCAL_DIR, 'paralympic-katiepress', 'medal_athlete.csv'), low_memory=False)

# Filter to USA — check what field works
print(f"Total rows: {len(katie):,}")
print(f"\nCountry-like columns:")
for col in ['npc', 'npc_new', 'npc_name', 'games_country']:
    vals = katie[col].dropna().unique()[:10]
    print(f"  {col}: {vals}")

# Filter to USA
katie_usa = katie[katie['npc_name'].astype(str).str.contains('United States', case=False, na=False) |
                   katie['npc'].astype(str).str.upper().eq('USA') |
                   katie['npc_new'].astype(str).str.upper().eq('USA')].copy()

print(f"\nUSA medal rows: {len(katie_usa):,}")
print(f"Unique athlete names: {katie_usa['athlete_name'].nunique():,}")
print(f"Year range: {katie_usa['games_year'].min()} — {katie_usa['games_year'].max()}")
print(f"Seasons: {sorted(katie_usa['games_season'].dropna().unique())}")
print(f"\nSports ({katie_usa['sport'].nunique()} unique): {sorted(katie_usa['sport'].dropna().unique()[:15])}")

# Extract classification from event
katie_usa['classification_code'] = katie_usa['event'].apply(extract_classifications)
cc_k = katie_usa['classification_code']
print(f"\nClassification extraction from 'event':")
print(f"  Extracted: {cc_k.notna().sum()} / {len(katie_usa)} ({cc_k.notna().sum()/len(katie_usa)*100:.1f}%)")
print(f"  Sample:")
for _, row in katie_usa[cc_k.notna()][['athlete_name', 'event', 'classification_code']].head(5).iterrows():
    print(f"    {row['athlete_name']:30s} event={str(row['event'])[:50]:50s} → {row['classification_code']}")

# Check for events with NO classification (team sports, early games, etc.)
no_class = katie_usa[cc_k.isna()]
print(f"\n  No classification extracted: {len(no_class)} rows")
print(f"  Sports without classification:")
print(no_class['sport'].value_counts().head(10).to_string())
print(f"\n  Sample unextracted events:")
print(no_class[['event', 'sport']].drop_duplicates().head(10).to_string())

In [ ]:
# ── Phase 2, Step 4: Deduplicate katiepress to unique athletes ────
# Each row is a medal record — aggregate to unique athletes

def concat_unique(s):
    vals = s.dropna().unique()
    return ','.join(sorted(str(v) for v in vals)) if len(vals) > 0 else None

katie_athletes = katie_usa.groupby('athlete_name').agg(
    first_games_year=('games_year', 'min'),
    last_games_year=('games_year', 'max'),
    games_count=('games_year', 'nunique'),
    games_seasons=('games_season', lambda x: ','.join(sorted(x.dropna().unique()))),
    sports=('sport', lambda x: ','.join(sorted(x.dropna().unique()))),
    primary_sport=('sport', most_common),
    classification_codes=('classification_code', lambda x: concat_unique(x)),
    gold_count=('medal', lambda x: (x.astype(str).str.lower() == 'gold').sum()),
    silver_count=('medal', lambda x: (x.astype(str).str.lower() == 'silver').sum()),
    bronze_count=('medal', lambda x: (x.astype(str).str.lower() == 'bronze').sum()),
).reset_index()

katie_athletes['total_medals'] = katie_athletes['gold_count'] + katie_athletes['silver_count'] + katie_athletes['bronze_count']

# Pick most specific classification code (prefer single over compound)
def pick_primary_classification(codes_str):
    if pd.isna(codes_str):
        return None
    codes = [c.strip() for c in codes_str.split(',') if c.strip()]
    codes = [c for c in codes if c.lower() != 'none']
    return codes[0] if codes else None

katie_athletes['classification_code'] = katie_athletes['classification_codes'].apply(pick_primary_classification)

print(f"Katiepress deduplicated: {len(katie_usa):,} medal rows → {len(katie_athletes):,} unique athletes")
print(f"  With classification: {katie_athletes['classification_code'].notna().sum():,}")
print(f"  Year range: {katie_athletes['first_games_year'].min()} — {katie_athletes['last_games_year'].max()}")
print(f"  Seasons: {katie_athletes['games_seasons'].value_counts().to_string()}")
print(f"\n  Medal distribution:")
print(f"    Gold:   {katie_athletes['gold_count'].sum():,}")
print(f"    Silver: {katie_athletes['silver_count'].sum():,}")
print(f"    Bronze: {katie_athletes['bronze_count'].sum():,}")
print(f"\n  Multi-Games athletes: {(katie_athletes['games_count'] > 1).sum():,}")
print(f"\nSample:")
print(katie_athletes[['athlete_name', 'primary_sport', 'classification_code',
                       'games_count', 'total_medals', 'first_games_year',
                       'last_games_year', 'games_seasons']].head(10).to_string())

In [ ]:
# ── Phase 2, Step 5: Overlap check across Paralympic sources ──────
# How many athletes appear in multiple sources?

def norm_para_name(n):
    if pd.isna(n): return ''
    s = str(n).upper().strip()
    s = re.sub(r'[^A-Z\s]', '', s)
    return ' '.join(sorted(s.split()))

tokyo_names = set(tokyo_usa['name'].apply(norm_para_name))
paris_names = set(paris_para_usa['name'].apply(norm_para_name))
katie_names = set(katie_athletes['athlete_name'].apply(norm_para_name))

print(f"Unique normalized names per source:")
print(f"  Tokyo 2020:       {len(tokyo_names):,}")
print(f"  Paris 2024:       {len(paris_names):,}")
print(f"  Katiepress hist:  {len(katie_names):,}")

print(f"\nOverlap:")
print(f"  Tokyo ∩ Paris:    {len(tokyo_names & paris_names):,}")
print(f"  Tokyo ∩ Katie:    {len(tokyo_names & katie_names):,}")
print(f"  Paris ∩ Katie:    {len(paris_names & katie_names):,}")
print(f"  All three:        {len(tokyo_names & paris_names & katie_names):,}")

all_para = tokyo_names | paris_names | katie_names
print(f"\n  Union (unique):   {len(all_para):,}")

# Note: katiepress names are abbreviated (e.g., "WINTERS A") so overlap will be low
# Show format differences
print(f"\nName format samples:")
print(f"  Tokyo:  {list(tokyo_usa['name'].head(3))}")
print(f"  Paris:  {list(paris_para_usa['name'].head(3))}")
print(f"  Katie:  {list(katie_athletes['athlete_name'].head(3))}")

In [ ]:
# ── PHASE 2 QC REPORT ─────────────────────────────────────────────
print('=' * 70)
print('PHASE 2 QC: PARALYMPIC ATHLETES')
print('=' * 70)

print(f'\n📊 SOURCE COUNTS')
print(f'  Tokyo 2020 USA athletes:   {len(tokyo_usa):,}')
print(f'  Paris 2024 USA athletes:   {len(paris_para_usa):,}')
print(f'  Katiepress USA athletes:   {len(katie_athletes):,} (deduplicated from {len(katie_usa):,} medal rows)')
print(f'  Estimated union:           {len(all_para):,} (name-based, expect noise)')

print(f'\n📅 TEMPORAL RANGE')
print(f'  Katiepress: {katie_athletes["first_games_year"].min()} — {katie_athletes["last_games_year"].max()}')
print(f'  Tokyo:      2020 (2021 actual)')
print(f'  Paris:      2024')

print(f'\n🏷️ CLASSIFICATION CODES')
print(f'  Tokyo (sport_class):       {tokyo_usa["sport_class"].notna().sum()} / {len(tokyo_usa)} ({tokyo_usa["sport_class"].notna().sum()/len(tokyo_usa)*100:.1f}%)')
paris_cc = paris_para_usa['classification_code'].notna().sum()
print(f'  Paris (extracted):         {paris_cc} / {len(paris_para_usa)} ({paris_cc/len(paris_para_usa)*100:.1f}%)')
katie_cc = katie_athletes['classification_code'].notna().sum()
print(f'  Katiepress (extracted):    {katie_cc} / {len(katie_athletes)} ({katie_cc/len(katie_athletes)*100:.1f}%)')

print(f'\n📏 PHYSICAL ATTRIBUTES (Paris 2024 only)')
h = (paris_para_usa['height'].notna() & (paris_para_usa['height'] > 0)).sum()
w = (paris_para_usa['weight'].notna() & (paris_para_usa['weight'] > 0)).sum()
print(f'  Height: {h} / {len(paris_para_usa)} ({h/len(paris_para_usa)*100:.1f}%)')
print(f'  Weight: {w} / {len(paris_para_usa)} ({w/len(paris_para_usa)*100:.1f}%)')

print(f'\n📝 BIO FIELDS (Paris 2024 only)')
bio_fields = ['reason', 'hero', 'philosophy', 'other_sports', 'coach', 'hobbies']
for field in bio_fields:
    if field in paris_para_usa.columns:
        n = paris_para_usa[field].notna().sum()
        print(f'  {field:25s} {n:>4} / {len(paris_para_usa)} ({n/len(paris_para_usa)*100:.1f}%)')

print(f'\n🏅 MEDAL TOTALS (katiepress historical)')
print(f'  Gold:   {katie_athletes["gold_count"].sum():,}')
print(f'  Silver: {katie_athletes["silver_count"].sum():,}')
print(f'  Bronze: {katie_athletes["bronze_count"].sum():,}')
print(f'  Total:  {katie_athletes["total_medals"].sum():,}')

print(f'\n🔀 OVERLAP')
print(f'  Tokyo ∩ Paris:    {len(tokyo_names & paris_names):,}')
print(f'  Tokyo ∩ Katie:    {len(tokyo_names & katie_names):,}')
print(f'  Paris ∩ Katie:    {len(paris_names & katie_names):,}')

print(f'\n📋 SAMPLE ATHLETES (5 from each source):')
print(f'\n  -- Tokyo 2020 --')
print(tokyo_usa[['name', 'gender', 'discipline', 'sport_class']].head(5).to_string())
print(f'\n  -- Paris 2024 --')
print(paris_para_usa[['name', 'gender', 'disciplines', 'classification_code']].head(5).to_string())
print(f'\n  -- Katiepress (deduplicated) --')
print(katie_athletes[['athlete_name', 'primary_sport', 'classification_code',
                        'games_count', 'total_medals']].head(5).to_string())

print(f'\n{"=" * 70}')
print('Paste this output back for review before proceeding to Phase 3.')
print('=' * 70)

---
## Phase 3: Unify Athletes Table

Merge Olympic and Paralympic athletes into a single `team_usa_athletes` table.

**Merge strategy:**
- **Olympic backbone** (10,332) + **Paris 2024 Olympic new arrivals** (~619, pending better name matching)
- **Tokyo 2020 Paralympic** (252) + **Paris 2024 Paralympic** (220) → merge on name, 108 overlap expected
- **Katiepress historical Paralympic** (1,169) → standalone cohort, no cross-matching
- Stack all into unified schema, generate athlete_id

**Key challenges:** Name format normalization, dedup across sources, column harmonization

In [ ]:
# ── Phase 3, Step 1: Merge Tokyo + Paris Paralympic ───────────────
# Tokyo has clean sport_class, Paris has rich bio fields
# For overlapping athletes: keep both, merge attributes

def norm_para(n):
    if pd.isna(n): return ''
    s = re.sub(r'[^A-Za-z\s]', '', str(n)).upper().strip()
    return ' '.join(sorted(s.split()))

tokyo_usa['_norm'] = tokyo_usa['name'].apply(norm_para)
paris_para_usa['_norm'] = paris_para_usa['name'].apply(norm_para)

# Merge on normalized name
para_merged = paris_para_usa.merge(
    tokyo_usa[['_norm', 'sport_class', 'discipline']].rename(
        columns={'sport_class': 'tokyo_sport_class', 'discipline': 'tokyo_discipline'}),
    on='_norm', how='outer', indicator=True
)

print(f"Paralympic merge results:")
print(para_merged['_merge'].value_counts().to_string())

# For athletes in both: prefer Paris bio fields + Tokyo sport_class
# For Paris-only: use extracted classification
# For Tokyo-only: need to pull full Tokyo row

# Handle Tokyo-only athletes (not in Paris)
tokyo_only_norms = set(para_merged[para_merged['_merge'] == 'right_only']['_norm'])
tokyo_only = tokyo_usa[tokyo_usa['_norm'].isin(tokyo_only_norms)].copy()

print(f"\nTokyo-only athletes: {len(tokyo_only)}")
print(f"Paris-only athletes: {(para_merged['_merge'] == 'left_only').sum()}")
print(f"In both:             {(para_merged['_merge'] == 'both').sum()}")

# Build unified Paralympic recent (Tokyo + Paris)
# Start with Paris athletes (richer data)
paris_cols = {
    'name': 'name', 'gender': 'gender', 'birth_date': 'birth_date',
    'birth_place': 'birth_place', 'birth_country': 'birth_country',
    'height': 'height_cm', 'weight': 'weight_kg',
    'disciplines': 'primary_sport', 'classification_code': 'classification_code',
    'events': 'events_raw',
    'hobbies': 'hobbies', 'occupation': 'occupation', 'education': 'education',
    'reason': 'reason', 'hero': 'hero', 'philosophy': 'philosophy',
    'other_sports': 'other_sports', 'coach': 'coach',
    'tokyo_sport_class': 'tokyo_sport_class', '_norm': '_norm'
}

# Filter to Paris athletes (both + left_only)
paris_rows = para_merged[para_merged['_merge'].isin(['both', 'left_only'])].copy()
available_cols = {k: v for k, v in paris_cols.items() if k in paris_rows.columns}
para_recent = paris_rows[list(available_cols.keys())].rename(columns=available_cols)

# For athletes in both sources, prefer Tokyo sport_class over extracted classification
has_tokyo_class = para_recent['tokyo_sport_class'].notna()
para_recent.loc[has_tokyo_class, 'classification_code'] = para_recent.loc[has_tokyo_class, 'tokyo_sport_class']
print(f"\nUpgraded {has_tokyo_class.sum()} athletes with Tokyo sport_class")

# Add Tokyo-only athletes
tokyo_only_std = tokyo_only.rename(columns={
    'name': 'name', 'gender': 'gender', 'birth_date': 'birth_date',
    'birth_place': 'birth_place', 'birth_country': 'birth_country',
    'discipline': 'primary_sport', 'sport_class': 'classification_code'
}).copy()
tokyo_only_std['height_cm'] = None
tokyo_only_std['weight_kg'] = None

# Stack
common_cols = ['name', 'gender', 'birth_date', 'birth_place', 'birth_country',
               'height_cm', 'weight_kg', 'primary_sport', 'classification_code', '_norm']
bio_cols = ['hobbies', 'occupation', 'education', 'reason', 'hero',
            'philosophy', 'other_sports', 'coach', 'events_raw']
for col in common_cols + bio_cols:
    if col not in para_recent.columns:
        para_recent[col] = None
    if col not in tokyo_only_std.columns:
        tokyo_only_std[col] = None

para_recent_all = pd.concat([
    para_recent[common_cols + bio_cols],
    tokyo_only_std[common_cols + bio_cols]
], ignore_index=True)

para_recent_all['games_type'] = 'Paralympic'
para_recent_all['source'] = 'piterfm'

print(f"\nCombined recent Paralympic athletes: {len(para_recent_all)}")
print(f"  With classification: {para_recent_all['classification_code'].notna().sum()}")
print(f"  With bio fields:     {para_recent_all['reason'].notna().sum()}")

In [ ]:
# ── Phase 3, Step 2: Prepare katiepress historical athletes ───────
# Clean name artifacts, standardize columns

def clean_katie_name(n):
    if pd.isna(n): return None
    s = str(n).strip()
    s = re.sub(r'^[^A-Za-z]+', '', s)  # strip leading punctuation
    s = s.strip()
    return s if len(s) >= 3 else None

katie_athletes['name_clean'] = katie_athletes['athlete_name'].apply(clean_katie_name)
before = len(katie_athletes)
katie_clean = katie_athletes[katie_athletes['name_clean'].notna()].copy()
print(f"Katiepress name cleanup: {before} → {len(katie_clean)} (dropped {before - len(katie_clean)} bad names)")

# Show what was dropped
dropped = katie_athletes[katie_athletes['name_clean'].isna()]
if len(dropped) > 0:
    print(f"  Dropped names: {dropped['athlete_name'].tolist()[:10]}")

# Standardize to unified schema
katie_std = katie_clean.rename(columns={
    'name_clean': 'name',
    'classification_code': 'classification_code',
    'primary_sport': 'primary_sport',
    'first_games_year': 'first_games_year',
    'last_games_year': 'last_games_year',
    'games_count': 'games_count',
    'games_seasons': 'games_seasons',
    'gold_count': 'gold_count',
    'silver_count': 'silver_count',
    'bronze_count': 'bronze_count',
    'total_medals': 'total_medals'
}).copy()

katie_std['gender'] = None  # katiepress doesn't have gender
katie_std['birth_date'] = None
katie_std['birth_place'] = None
katie_std['birth_country'] = None
katie_std['height_cm'] = None
katie_std['weight_kg'] = None
katie_std['games_type'] = 'Paralympic'
katie_std['source'] = 'katiepress'

print(f"\nKatiepress standardized: {len(katie_std)} athletes")
print(f"  With classification: {katie_std['classification_code'].notna().sum()}")
print(f"  Year range: {katie_std['first_games_year'].min()} — {katie_std['last_games_year'].max()}")
print(f"  Season split:\n{katie_std['games_seasons'].value_counts().to_string()}")

In [ ]:
# ── Phase 3, Step 3: Paris 2024 Olympic — improved name matching ──
# Paris uses "LAST First" format, keithgalli uses "First•Last"
# Strategy: normalize both to sorted uppercase tokens + cross-check birth_date

def norm_olympic(n):
    if pd.isna(n): return ''
    s = str(n).replace('•', ' ').replace('-', ' ')
    s = re.sub(r'[^A-Za-z\s]', '', s).upper().strip()
    return ' '.join(sorted(s.split()))

# Re-normalize with bullet handling
olympic_athletes['_norm'] = olympic_athletes['name'].apply(norm_olympic)
paris_usa['_norm'] = paris_usa['name'].apply(norm_olympic)

# Pass 1: exact normalized name match
matched_name = paris_usa['_norm'].isin(olympic_athletes['_norm'])
print(f"Paris 2024 Olympic name matching:")
print(f"  Pass 1 (normalized name): {matched_name.sum()} / {len(paris_usa)}")

# Pass 2: for unmatched, try name + birth_date
unmatched_paris = paris_usa[~matched_name].copy()
if 'birth_date' in unmatched_paris.columns and 'birth_date' in olympic_athletes.columns:
    # Normalize birth dates
    olympic_athletes['_bd'] = pd.to_datetime(olympic_athletes['birth_date'], errors='coerce').dt.strftime('%Y-%m-%d')
    unmatched_paris['_bd'] = pd.to_datetime(unmatched_paris['birth_date'], errors='coerce').dt.strftime('%Y-%m-%d')

    # Match on birth_date alone for remaining
    bd_matched = unmatched_paris['_bd'].isin(olympic_athletes['_bd'].dropna())
    print(f"  Unmatched with valid birth_date: {unmatched_paris['_bd'].notna().sum()}")
    print(f"  Pass 2 (birth_date exists in backbone): {bd_matched.sum()} (note: not unique, just a signal)")

# Pass 3: fuzzy match on last name + birth year
unmatched_paris['_last'] = unmatched_paris['name'].apply(
    lambda n: str(n).split()[0].upper() if pd.notna(n) else '')
unmatched_paris['_by'] = pd.to_datetime(unmatched_paris['birth_date'], errors='coerce').dt.year

olympic_athletes['_last'] = olympic_athletes['name'].apply(
    lambda n: str(n).replace('•', ' ').split()[-1].upper() if pd.notna(n) else '')
olympic_athletes['_by'] = pd.to_datetime(olympic_athletes['birth_date'], errors='coerce').dt.year

last_year_match = unmatched_paris.merge(
    olympic_athletes[['_last', '_by']].drop_duplicates(),
    on=['_last', '_by'], how='inner'
)
print(f"  Pass 3 (last name + birth year): {len(last_year_match)} additional matches")

total_matched = matched_name.sum() + len(last_year_match)
truly_new = len(paris_usa) - total_matched
print(f"\n  Total matched:   {total_matched}")
print(f"  Truly new 2024:  {truly_new}")

# Identify the truly new athletes
matched_norms = set(olympic_athletes['_norm'])
pass3_norms = set(last_year_match['_norm'])
paris_new = paris_usa[(~paris_usa['_norm'].isin(matched_norms)) &
                       (~paris_usa['_norm'].isin(pass3_norms))].copy()

print(f"\nNew Paris 2024 athletes — discipline distribution:")
if 'disciplines' in paris_new.columns:
    print(paris_new['disciplines'].value_counts().head(15).to_string())

# Clean up temp columns
for df in [olympic_athletes, paris_usa, unmatched_paris]:
    for col in ['_norm', '_bd', '_last', '_by']:
        if col in df.columns:
            df.drop(columns=[col], inplace=True, errors='ignore')

In [ ]:
# ── Phase 3, Step 4: Build rows for genuinely new Paris 2024 athletes ──

paris_new_std = paris_new.rename(columns={
    'name': 'name', 'gender': 'gender', 'birth_date': 'birth_date',
    'birth_place': 'birth_place', 'birth_country': 'birth_country',
    'disciplines': 'primary_sport'
}).copy()

# Handle height/weight (only keep non-zero)
if 'height' in paris_new.columns:
    paris_new_std['height_cm'] = paris_new['height'].where(paris_new['height'] > 0, None)
else:
    paris_new_std['height_cm'] = None

if 'weight' in paris_new.columns:
    paris_new_std['weight_kg'] = paris_new['weight'].where(paris_new['weight'] > 0, None)
else:
    paris_new_std['weight_kg'] = None

paris_new_std['games_type'] = 'Olympic'
paris_new_std['classification_code'] = None
paris_new_std['first_games_year'] = 2024
paris_new_std['last_games_year'] = 2024
paris_new_std['games_count'] = 1
paris_new_std['games_types'] = 'Summer'
paris_new_std['gold_count'] = 0
paris_new_std['silver_count'] = 0
paris_new_std['bronze_count'] = 0
paris_new_std['total_medals'] = 0
paris_new_std['source'] = 'paris2024'

print(f"New Paris 2024 Olympic athletes to add: {len(paris_new_std)}")
h = (paris_new_std['height_cm'].notna()).sum()
w = (paris_new_std['weight_kg'].notna()).sum()
print(f"  With height: {h}  With weight: {w}")

In [ ]:
# ── Phase 3, Step 5: Stack everything into team_usa_athletes ──────

# Define unified columns
unified_cols = [
    'name', 'gender', 'birth_date', 'birth_place', 'birth_country',
    'height_cm', 'weight_kg', 'games_type', 'classification_code',
    'primary_sport', 'first_games_year', 'last_games_year', 'games_count',
    'gold_count', 'silver_count', 'bronze_count', 'total_medals', 'source'
]
bio_cols = ['hobbies', 'occupation', 'education', 'reason', 'hero',
            'philosophy', 'other_sports', 'coach']

# Prep Olympic backbone
olympic_for_stack = olympic_athletes.copy()
olympic_for_stack['classification_code'] = None
olympic_for_stack['source'] = 'keithgalli'
# Rename games_types → games_seasons for consistency
if 'games_types' in olympic_for_stack.columns:
    olympic_for_stack['games_seasons'] = olympic_for_stack['games_types']

# Ensure all dataframes have all columns
all_dfs = {
    'Olympic backbone': olympic_for_stack,
    'Paris 2024 Olympic new': paris_new_std,
    'Paralympic recent (Tokyo+Paris)': para_recent_all,
    'Paralympic historical (katiepress)': katie_std
}

for label, df in all_dfs.items():
    for col in unified_cols + bio_cols:
        if col not in df.columns:
            df[col] = None
    print(f"{label:45s} → {len(df):>6,} rows")

# Stack
team_usa_athletes = pd.concat(
    [df[unified_cols + bio_cols] for df in all_dfs.values()],
    ignore_index=True
)

print(f"\n{'=' * 50}")
print(f"STACKED TOTAL: {len(team_usa_athletes):,} athletes")
print(f"  Olympic:    {(team_usa_athletes['games_type'] == 'Olympic').sum():,}")
print(f"  Paralympic: {(team_usa_athletes['games_type'] == 'Paralympic').sum():,}")

In [ ]:
# ── Phase 3, Step 6: Generate athlete_id and dedup ────────────────
import hashlib

def make_athlete_id(row):
    """Generate stable ID from name + birth_date + games_type."""
    parts = [
        str(row.get('name', '')).upper().strip(),
        str(row.get('birth_date', '')).strip(),
        str(row.get('games_type', '')).strip()
    ]
    key = '|'.join(parts)
    return hashlib.md5(key.encode()).hexdigest()[:12]

team_usa_athletes['athlete_id'] = team_usa_athletes.apply(make_athlete_id, axis=1)

# Check for ID collisions
dupes = team_usa_athletes[team_usa_athletes['athlete_id'].duplicated(keep=False)]
print(f"Athlete ID collisions: {dupes['athlete_id'].nunique()} IDs affecting {len(dupes)} rows")

if len(dupes) > 0:
    print(f"\nSample collisions:")
    for aid in dupes['athlete_id'].unique()[:5]:
        subset = dupes[dupes['athlete_id'] == aid]
        print(f"\n  ID: {aid}")
        print(subset[['name', 'birth_date', 'games_type', 'source', 'primary_sport']].to_string())

# For true duplicates (same person from overlapping sources): keep the row with most data
def completeness_score(row):
    return row.notna().sum()

team_usa_athletes['_completeness'] = team_usa_athletes.apply(completeness_score, axis=1)
team_usa_athletes = team_usa_athletes.sort_values('_completeness', ascending=False)
before_dedup = len(team_usa_athletes)
team_usa_athletes = team_usa_athletes.drop_duplicates(subset='athlete_id', keep='first')
team_usa_athletes = team_usa_athletes.drop(columns=['_completeness'])

print(f"\nDedup: {before_dedup:,} → {len(team_usa_athletes):,} (removed {before_dedup - len(team_usa_athletes):,})")

In [ ]:
# ── Phase 3, Step 7: Checkpoint to GCS ────────────────────────────
checkpoint_path = '/tmp/team_usa_athletes.csv'
team_usa_athletes.to_csv(checkpoint_path, index=False)

!gsutil cp {checkpoint_path} gs://class-demo/team-usa/processed/team_usa_athletes.csv

print(f"✅ Checkpoint saved: gs://class-demo/team-usa/processed/team_usa_athletes.csv")
print(f"   Rows: {len(team_usa_athletes):,}")

In [ ]:
# ── PHASE 3 QC REPORT ─────────────────────────────────────────────
print('=' * 70)
print('PHASE 3 QC: UNIFIED TEAM USA ATHLETES')
print('=' * 70)

print(f'\n📊 COUNTS')
print(f'  Total athletes:     {len(team_usa_athletes):,}')
print(f'  Olympic:            {(team_usa_athletes["games_type"] == "Olympic").sum():,}')
print(f'  Paralympic:         {(team_usa_athletes["games_type"] == "Paralympic").sum():,}')

print(f'\n📦 SOURCE BREAKDOWN')
print(team_usa_athletes['source'].value_counts().to_string())

print(f'\n📅 TEMPORAL RANGE')
oly = team_usa_athletes[team_usa_athletes['games_type'] == 'Olympic']
para = team_usa_athletes[team_usa_athletes['games_type'] == 'Paralympic']
print(f'  Olympic:    {oly["first_games_year"].min()} — {oly["last_games_year"].max()}')
print(f'  Paralympic: {para["first_games_year"].min()} — {para["last_games_year"].max()}')

print(f'\n🏷️ CLASSIFICATION CODES')
has_class = team_usa_athletes['classification_code'].notna()
print(f'  Total with classification: {has_class.sum():,}')
print(f'  Paralympic with classification: {(has_class & (team_usa_athletes["games_type"] == "Paralympic")).sum():,} / {len(para):,}')
print(f'  Olympic with classification (should be 0): {(has_class & (team_usa_athletes["games_type"] == "Olympic")).sum():,}')

print(f'\n📏 PHYSICAL ATTRIBUTES')
h = (team_usa_athletes['height_cm'].notna() & (team_usa_athletes['height_cm'] > 0)).sum()
w = (team_usa_athletes['weight_kg'].notna() & (team_usa_athletes['weight_kg'] > 0)).sum()
print(f'  Height: {h:,} / {len(team_usa_athletes):,} ({h/len(team_usa_athletes)*100:.1f}%)')
print(f'  Weight: {w:,} / {len(team_usa_athletes):,} ({w/len(team_usa_athletes)*100:.1f}%)')
print(f'  Olympic height:    {(oly["height_cm"].notna() & (oly["height_cm"] > 0)).sum():,} / {len(oly):,}')
print(f'  Paralympic height: {(para["height_cm"].notna() & (para["height_cm"] > 0)).sum():,} / {len(para):,}')

print(f'\n🏅 MEDALS')
print(f'  Total medals:       {team_usa_athletes["total_medals"].sum():,.0f}')
print(f'  Athletes with 1+:   {(team_usa_athletes["total_medals"] > 0).sum():,}')
print(f'  Olympic medalists:  {(oly["total_medals"] > 0).sum():,}')
print(f'  Paralympic medalists: {(para["total_medals"] > 0).sum():,}')

print(f'\n📝 BIO FIELDS (from Paris 2024 Paralympic)')
for field in ['reason', 'hero', 'philosophy', 'other_sports', 'coach']:
    if field in team_usa_athletes.columns:
        n = team_usa_athletes[field].notna().sum()
        print(f'  {field:25s} {n:>5} / {len(team_usa_athletes):,}')

print(f'\n⚧ GENDER')
print(team_usa_athletes['gender'].value_counts(dropna=False).to_string())

print(f'\n🔑 ATHLETE ID')
print(f'  Unique IDs: {team_usa_athletes["athlete_id"].nunique():,}')
print(f'  Any remaining dupes: {team_usa_athletes["athlete_id"].duplicated().sum()}')

print(f'\n📋 SAMPLE ATHLETES (3 Olympic, 3 Paralympic):')
sample_cols = ['athlete_id', 'name', 'games_type', 'classification_code',
               'primary_sport', 'games_count', 'total_medals', 'source']
print(oly[sample_cols].dropna(subset=['games_count']).sample(3, random_state=42).to_string())
print()
print(para[sample_cols].sample(3, random_state=42).to_string())

print(f'\n{"=" * 70}')
print('Paste this output back for review before proceeding to Phase 4.')
print('=' * 70)

---
## Phase 4: Unified Results Table + Athlete Backfill

Build `team_usa_results` from all medal/event sources, then backfill career stats into the athletes table.

**Sources:**
- `keithgalli/results.csv` → Olympic all participations 1896–2022 (21,353 USA rows)
- `olympic-paris2024/medallists.csv` → Olympic medal records 2024
- `piterfm/2020_Tokyo/` + `2024_Paris/` medal files → Paralympic 2020 + 2024
- `katiepress/medal_athlete.csv` → Paralympic medals 1960–2018 (already loaded)

**Backfill strategy:** Recompute career stats from the unified results table as single source of truth. Also infer gender for katiepress athletes from event names.

In [ ]:
# ── Phase 4, Step 1: Standardize keithgalli Olympic results ──────
print(f"keithgalli results columns: {list(usa_results.columns)}")
print(f"USA rows: {len(usa_results):,}\n")

# Create lookup: keithgalli numeric athlete_id → unified hash ID
# olympic_athletes still has the original keithgalli athlete_id
kg_id_lookup = olympic_athletes[['athlete_id', 'name', 'birth_date']].copy()

# Compute unified hash (same formula as Cell 21)
kg_id_lookup['unified_id'] = kg_id_lookup.apply(
    lambda r: hashlib.md5(
        f"{str(r['name']).upper().strip()}|{str(r['birth_date']).strip()}|Olympic".encode()
    ).hexdigest()[:12], axis=1)

# Verify
in_athletes = kg_id_lookup['unified_id'].isin(team_usa_athletes['athlete_id'])
print(f"ID verification: {in_athletes.sum():,} / {len(kg_id_lookup):,} found in team_usa_athletes")

# Build mapping dicts
kg_to_unified = dict(zip(kg_id_lookup['athlete_id'], kg_id_lookup['unified_id']))
kg_to_name = dict(zip(kg_id_lookup['athlete_id'], kg_id_lookup['name']))

# Determine sport column
sport_col = 'sport' if 'sport' in usa_results.columns else 'discipline'

# Standardize
results_keithgalli = pd.DataFrame({
    'athlete_id': usa_results['athlete_id'].map(kg_to_unified),
    'athlete_name': usa_results['athlete_id'].map(kg_to_name),
    'games_year': usa_results['year'].fillna(0).astype(int),
    'games_season': usa_results['type'],
    'games_type': 'Olympic',
    'sport': usa_results[sport_col],
    'discipline': usa_results['discipline'],
    'event': usa_results['event'],
    'classification_code': None,
    'medal': usa_results['medal'],
    'source': 'keithgalli'
})

print(f"\nStandardized: {len(results_keithgalli):,} rows")
print(f"  Mapped to unified ID: {results_keithgalli['athlete_id'].notna().sum():,}")
print(f"  Unmapped: {results_keithgalli['athlete_id'].isna().sum()}")
print(f"  Year range: {results_keithgalli['games_year'].min()} — {results_keithgalli['games_year'].max()}")
print(f"  Medal dist:\n{results_keithgalli['medal'].value_counts(dropna=False).head().to_string()}")

In [ ]:
# ── Phase 4, Step 2: Paris 2024 Olympic medal results ────────────
p2024_dir = os.path.join(LOCAL_DIR, 'olympic-paris2024')
p2024_files = [f for f in os.listdir(p2024_dir) if f.endswith('.csv')]
print(f"Paris 2024 CSV files: {p2024_files}")

# Load medallists (prefer over medals — one row per athlete)
med_file = 'medallists.csv' if 'medallists.csv' in p2024_files else 'medals.csv'
paris_med_raw = pd.read_csv(os.path.join(p2024_dir, med_file), low_memory=False)
print(f"\n{med_file}: {len(paris_med_raw):,} total rows")
print(f"Columns: {list(paris_med_raw.columns)}")

# Find USA filter column
usa_col = None
for c in paris_med_raw.columns:
    if paris_med_raw[c].astype(str).str.upper().eq('USA').any() and ('country' in c.lower() or 'code' in c.lower()):
        usa_col = c
        break
if usa_col is None:
    for c in paris_med_raw.columns:
        if paris_med_raw[c].astype(str).str.upper().eq('USA').any():
            usa_col = c
            break

paris_med_usa = paris_med_raw[paris_med_raw[usa_col].astype(str).str.upper().eq('USA')].copy()
print(f"USA rows (via '{usa_col}'): {len(paris_med_usa):,}")

# Identify columns
name_col = next((c for c in ['name', 'athlete_name', 'Name', 'athlete'] if c in paris_med_usa.columns), None)
medal_col = next((c for c in ['medal_type', 'medal_code', 'medal', 'Medal'] if c in paris_med_usa.columns), None)
disc_col = next((c for c in ['discipline', 'sport', 'Discipline', 'Sport'] if c in paris_med_usa.columns), None)
event_col = next((c for c in ['event', 'event_title', 'Event'] if c in paris_med_usa.columns), None)
print(f"Column mapping: name={name_col}, medal={medal_col}, discipline={disc_col}, event={event_col}")

# Print sample for verification
print(f"\nSample rows:")
show_cols = [c for c in [name_col, medal_col, disc_col, event_col] if c]
print(paris_med_usa[show_cols].head(3).to_string())

# Normalize medal values
medal_norm = {'Gold Medal': 'Gold', 'Silver Medal': 'Silver', 'Bronze Medal': 'Bronze',
              'GOLD': 'Gold', 'SILVER': 'Silver', 'BRONZE': 'Bronze',
              'Gold': 'Gold', 'Silver': 'Silver', 'Bronze': 'Bronze',
              '1': 'Gold', '2': 'Silver', '3': 'Bronze'}

results_paris2024 = pd.DataFrame({
    'athlete_id': None,
    'athlete_name': paris_med_usa[name_col] if name_col else None,
    'games_year': 2024,
    'games_season': 'Summer',
    'games_type': 'Olympic',
    'sport': paris_med_usa[disc_col] if disc_col else None,
    'discipline': paris_med_usa[disc_col] if disc_col else None,
    'event': paris_med_usa[event_col] if event_col else None,
    'classification_code': None,
    'medal': paris_med_usa[medal_col].map(medal_norm) if medal_col else None,
    'source': 'paris2024'
})

# Check for unmapped medal values
unmapped = results_paris2024['medal'].isna().sum()
print(f"\nStandardized: {len(results_paris2024):,} rows")
print(f"Medal dist:\n{results_paris2024['medal'].value_counts(dropna=False).to_string()}")
if unmapped > 0:
    raw_vals = paris_med_usa[medal_col].unique()
    print(f"⚠️ Unmapped medal values: {[v for v in raw_vals if v not in medal_norm]}")

In [ ]:
# ── Phase 4, Step 3: piterfm Paralympic medal results ─────────────
piterfm_dfs = []

for subdir, year in [('2020_Tokyo', 2020), ('2024_Paris', 2024)]:
    dirpath = os.path.join(LOCAL_DIR, 'paralympic-piterfm', subdir)
    files = [f for f in os.listdir(dirpath) if f.endswith('.csv')]
    print(f"\n{'=' * 50}")
    print(f"{subdir} files: {files}")

    # Find medal file (prefer medallists over medals)
    medal_file = None
    for candidate in ['medallists.csv', 'medals.csv']:
        if candidate in files:
            medal_file = candidate
            break
    if medal_file is None:
        print("  ⚠️ No medal file found!")
        continue

    df = pd.read_csv(os.path.join(dirpath, medal_file), low_memory=False)
    print(f"  {medal_file}: {len(df):,} rows, columns: {list(df.columns)}")

    # Filter to USA
    usa_col = None
    for c in df.columns:
        if df[c].astype(str).str.upper().eq('USA').any() and ('country' in c.lower() or 'code' in c.lower()):
            usa_col = c
            break
    if usa_col is None:
        for c in df.columns:
            if df[c].astype(str).str.contains('United States', case=False, na=False).any():
                usa_col = c
                break

    if usa_col is None:
        print("  ⚠️ Could not find USA column!")
        continue

    usa_df = df[df[usa_col].astype(str).str.contains('USA|United States', case=False, na=False)].copy()
    print(f"  USA rows (via '{usa_col}'): {len(usa_df):,}")

    if len(usa_df) == 0:
        continue

    # Map columns
    name_col = next((c for c in ['name', 'athlete_name', 'Name'] if c in usa_df.columns), None)
    medal_col = next((c for c in ['medal_type', 'medal_code', 'medal', 'Medal'] if c in usa_df.columns), None)
    disc_col = next((c for c in ['discipline', 'sport', 'Discipline'] if c in usa_df.columns), None)
    event_col = next((c for c in ['event', 'event_title', 'Event'] if c in usa_df.columns), None)
    class_col = next((c for c in ['sport_class', 'classification'] if c in usa_df.columns), None)

    medal_norm = {'Gold Medal': 'Gold', 'Silver Medal': 'Silver', 'Bronze Medal': 'Bronze',
                  'GOLD': 'Gold', 'SILVER': 'Silver', 'BRONZE': 'Bronze',
                  'Gold': 'Gold', 'Silver': 'Silver', 'Bronze': 'Bronze'}

    std = pd.DataFrame({
        'athlete_id': None,
        'athlete_name': usa_df[name_col] if name_col else None,
        'games_year': year,
        'games_season': 'Summer',
        'games_type': 'Paralympic',
        'sport': usa_df[disc_col] if disc_col else None,
        'discipline': usa_df[disc_col] if disc_col else None,
        'event': usa_df[event_col] if event_col else None,
        'classification_code': usa_df[class_col] if class_col else None,
        'medal': usa_df[medal_col].map(medal_norm) if medal_col else None,
        'source': f'piterfm_{year}'
    })

    piterfm_dfs.append(std)
    print(f"  Mapped: name={name_col}, medal={medal_col}, disc={disc_col}, event={event_col}, class={class_col}")
    print(f"  Medal dist: {std['medal'].value_counts(dropna=False).head().to_string()}")

results_piterfm = pd.concat(piterfm_dfs, ignore_index=True) if piterfm_dfs else pd.DataFrame()
print(f"\nTotal piterfm results: {len(results_piterfm):,}")

In [ ]:
# ── Phase 4, Step 4: katiepress results + gender extraction ───────

# Extract gender from event names (save for backfill)
def extract_gender(event_str):
    if pd.isna(event_str): return None
    s = str(event_str).lower()
    if 'women' in s or 'female' in s: return 'Female'
    if s.startswith('men') or "men's" in s or ' men ' in s or 'male' in s: return 'Male'
    if 'mixed' in s: return 'Mixed'
    return None

katie_usa['extracted_gender'] = katie_usa['event'].apply(extract_gender)

# Save gender mapping for athlete backfill
katie_gender = katie_usa.groupby('athlete_name')['extracted_gender'].agg(
    lambda x: x.dropna().mode().iloc[0] if len(x.dropna().mode()) > 0 else None
).reset_index()
katie_gender.columns = ['athlete_name', 'inferred_gender']
print(f"Gender extraction: {katie_gender['inferred_gender'].notna().sum()} / {len(katie_gender)} athletes")
print(katie_gender['inferred_gender'].value_counts(dropna=False).to_string())

# Standardize results
results_katiepress = pd.DataFrame({
    'athlete_id': None,
    'athlete_name': katie_usa['athlete_name'],
    'games_year': katie_usa['games_year'].astype(int),
    'games_season': katie_usa['games_season'],
    'games_type': 'Paralympic',
    'sport': katie_usa['sport'],
    'discipline': katie_usa['sport'],
    'event': katie_usa['event'],
    'classification_code': katie_usa['classification_code'],
    'medal': katie_usa['medal'].str.strip().str.title(),
    'source': 'katiepress'
})

print(f"\nKatiepress results: {len(results_katiepress):,}")
print(f"Medal dist:\n{results_katiepress['medal'].value_counts().to_string()}")

In [ ]:
# ── Phase 4, Step 5: Stack results + assign athlete_ids ───────────

# Improved name normalization: replace non-alpha with space (fixes bullet chars)
def norm_for_match(n):
    if pd.isna(n): return ''
    s = re.sub(r'[^A-Za-z\s]', ' ', str(n)).upper().strip()
    s = re.sub(r'\s+', ' ', s)
    return ' '.join(sorted(s.split()))

# Stack all results
team_usa_results = pd.concat([
    results_keithgalli,
    results_paris2024,
    results_piterfm,
    results_katiepress
], ignore_index=True)

print(f"Stacked results: {len(team_usa_results):,}")
print(f"  With athlete_id (keithgalli): {team_usa_results['athlete_id'].notna().sum():,}")
print(f"  Need ID assignment: {team_usa_results['athlete_id'].isna().sum():,}")

# Build athlete lookup: (normalized_name, games_type) → athlete_id
ath_lookup = team_usa_athletes[['athlete_id', 'name', 'games_type']].copy()
ath_lookup['_norm'] = ath_lookup['name'].apply(norm_for_match)
name_type_to_id = {}
for _, row in ath_lookup.iterrows():
    key = (row['_norm'], row['games_type'])
    if key not in name_type_to_id:
        name_type_to_id[key] = row['athlete_id']

# Assign IDs to unmatched results
needs_id = team_usa_results['athlete_id'].isna()
norm_names = team_usa_results.loc[needs_id, 'athlete_name'].apply(norm_for_match)
game_types = team_usa_results.loc[needs_id, 'games_type']

team_usa_results.loc[needs_id, 'athlete_id'] = [
    name_type_to_id.get((n, g)) for n, g in zip(norm_names, game_types)
]

# Report
matched = team_usa_results['athlete_id'].notna().sum()
unmatched = team_usa_results['athlete_id'].isna().sum()
print(f"\nAfter ID assignment:")
print(f"  Matched:   {matched:,}")
print(f"  Unmatched: {unmatched:,}")

if unmatched > 0:
    print(f"\n  Unmatched by source:")
    print(team_usa_results[team_usa_results['athlete_id'].isna()]['source'].value_counts().to_string())
    print(f"\n  Sample unmatched (first 10):")
    print(team_usa_results[team_usa_results['athlete_id'].isna()][
        ['athlete_name', 'games_type', 'games_year', 'event', 'source']].head(10).to_string())

# Duplicate check
dupes = team_usa_results.duplicated(subset=['athlete_id', 'games_year', 'games_type', 'event'], keep=False)
print(f"\nPotential duplicate results: {dupes.sum()} rows")
if dupes.sum() > 0 and dupes.sum() < 20:
    print(team_usa_results[dupes][['athlete_name', 'games_year', 'event', 'medal', 'source']].head(10).to_string())

In [ ]:
# ── Phase 4, Step 6: Backfill athletes table from results ────────

# 1. Recompute career stats from unified results (source of truth)
career_new = team_usa_results[team_usa_results['athlete_id'].notna()].groupby('athlete_id').agg(
    r_first_year=('games_year', 'min'),
    r_last_year=('games_year', 'max'),
    r_games_count=('games_year', 'nunique'),
    r_gold=('medal', lambda x: (x == 'Gold').sum()),
    r_silver=('medal', lambda x: (x == 'Silver').sum()),
    r_bronze=('medal', lambda x: (x == 'Bronze').sum()),
).reset_index()
career_new['r_total'] = career_new['r_gold'] + career_new['r_silver'] + career_new['r_bronze']

print(f"Career stats recomputed for {len(career_new):,} athletes from results")

# 2. Merge and replace
team_usa_athletes = team_usa_athletes.merge(career_new, on='athlete_id', how='left')

# Prefer results-derived (covers all sources) over original (single source)
field_map = {
    'first_games_year': 'r_first_year',
    'last_games_year': 'r_last_year',
    'games_count': 'r_games_count',
    'gold_count': 'r_gold',
    'silver_count': 'r_silver',
    'bronze_count': 'r_bronze',
    'total_medals': 'r_total'
}

for orig, derived in field_map.items():
    if derived in team_usa_athletes.columns:
        team_usa_athletes[orig] = team_usa_athletes[derived].fillna(team_usa_athletes[orig])

# Drop temp columns
r_cols = [c for c in team_usa_athletes.columns if c.startswith('r_')]
team_usa_athletes.drop(columns=r_cols, inplace=True)

# 3. Ensure piterfm athletes have participation data even without medals
for src_df, year, label in [(tokyo_usa, 2020, 'Tokyo 2020'), (paris_para_usa, 2024, 'Paris 2024')]:
    src_norms = set(src_df['name'].apply(norm_for_match))
    ath_norms = team_usa_athletes['name'].apply(norm_for_match)
    mask = ath_norms.isin(src_norms) & (team_usa_athletes['games_type'] == 'Paralympic')

    team_usa_athletes.loc[mask, 'first_games_year'] = team_usa_athletes.loc[mask, 'first_games_year'].apply(
        lambda x: min(x, year) if pd.notna(x) else float(year))
    team_usa_athletes.loc[mask, 'last_games_year'] = team_usa_athletes.loc[mask, 'last_games_year'].apply(
        lambda x: max(x, year) if pd.notna(x) else float(year))
    team_usa_athletes.loc[mask & team_usa_athletes['games_count'].isna(), 'games_count'] = 1
    print(f"  {label} participation ensured for {mask.sum()} athletes")

# Athletes in both Tokyo + Paris get games_count >= 2
both_norms = set(tokyo_usa['name'].apply(norm_for_match)) & set(paris_para_usa['name'].apply(norm_for_match))
both_mask = team_usa_athletes['name'].apply(norm_for_match).isin(both_norms) & \
            (team_usa_athletes['games_type'] == 'Paralympic')
team_usa_athletes.loc[both_mask, 'games_count'] = team_usa_athletes.loc[both_mask, 'games_count'].apply(
    lambda x: max(x, 2) if pd.notna(x) else 2)
print(f"  Dual-Games athletes (Tokyo+Paris): {both_mask.sum()}")

# 4. Fill NaN medals with 0 for athletes with known participation
has_participation = team_usa_athletes['games_count'].notna() & (team_usa_athletes['games_count'] > 0)
for col in ['gold_count', 'silver_count', 'bronze_count', 'total_medals']:
    before_na = team_usa_athletes[col].isna().sum()
    team_usa_athletes.loc[has_participation & team_usa_athletes[col].isna(), col] = 0
    filled = before_na - team_usa_athletes[col].isna().sum()
    if filled > 0:
        print(f"  {col}: filled {filled} NaNs → 0")

# 5. Gender backfill from katiepress event names
katie_gender['_norm'] = katie_gender['athlete_name'].apply(norm_for_match)
gender_map = dict(zip(katie_gender['_norm'], katie_gender['inferred_gender']))

team_usa_athletes['_norm'] = team_usa_athletes['name'].apply(norm_for_match)
needs_gender = team_usa_athletes['gender'].isna()
team_usa_athletes.loc[needs_gender, 'gender'] = team_usa_athletes.loc[needs_gender, '_norm'].map(gender_map)

gender_filled = needs_gender.sum() - team_usa_athletes['gender'].isna().sum()
team_usa_athletes.drop(columns=['_norm'], inplace=True)

print(f"\n  Gender backfill: {gender_filled} athletes updated")
print(f"  Remaining without gender: {team_usa_athletes['gender'].isna().sum()}")

print(f"\n✅ Backfill complete")

In [ ]:
# ── Phase 4, Step 6b: Cleanup ─────────────────────────────────────

# 1. Fix year=0 artifacts (NaN years that were filled with 0)
zero_year_results = (team_usa_results['games_year'] == 0).sum()
zero_year_athletes = (team_usa_athletes['first_games_year'] == 0).sum()

# In results: drop rows with year=0 (these have no usable data)
team_usa_results = team_usa_results[team_usa_results['games_year'] != 0].reset_index(drop=True)

# In athletes: set 0 back to NaN
for col in ['first_games_year', 'last_games_year']:
    team_usa_athletes.loc[team_usa_athletes[col] == 0, col] = None

print(f"Year=0 cleanup:")
print(f"  Results rows dropped: {zero_year_results}")
print(f"  Athletes years reset to NaN: {zero_year_athletes}")

# 2. Set "Mixed" gender to None (event descriptor, not athlete gender)
mixed_count = (team_usa_athletes['gender'] == 'Mixed').sum()
team_usa_athletes.loc[team_usa_athletes['gender'] == 'Mixed', 'gender'] = None
print(f"\n  'Mixed' gender reset to None: {mixed_count}")

print(f"\n✅ Cleanup complete")
print(f"  Results: {len(team_usa_results):,} rows")
print(f"  Athletes: {len(team_usa_athletes):,} rows")

In [ ]:
# ── Phase 4, Step 7: Checkpoint to GCS ────────────────────────────
ath_path = '/tmp/team_usa_athletes.csv'
team_usa_athletes.to_csv(ath_path, index=False)
!gsutil cp {ath_path} gs://class-demo/team-usa/processed/team_usa_athletes.csv

res_path = '/tmp/team_usa_results.csv'
team_usa_results.to_csv(res_path, index=False)
!gsutil cp {res_path} gs://class-demo/team-usa/processed/team_usa_results.csv

print(f"✅ Checkpoints saved:")
print(f"   Athletes: {len(team_usa_athletes):,} rows → gs://class-demo/team-usa/processed/team_usa_athletes.csv")
print(f"   Results:  {len(team_usa_results):,} rows → gs://class-demo/team-usa/processed/team_usa_results.csv")

In [ ]:
# ── PHASE 4 QC REPORT ─────────────────────────────────────────────
print('=' * 70)
print('PHASE 4 QC: UNIFIED RESULTS + BACKFILLED ATHLETES')
print('=' * 70)

print(f'\n{"─" * 40}')
print(f'RESULTS TABLE')
print(f'{"─" * 40}')

print(f'\n📊 COUNTS')
print(f'  Total result rows:     {len(team_usa_results):,}')
print(f'  With athlete_id:       {team_usa_results["athlete_id"].notna().sum():,}')
print(f'  Orphan (no ID):        {team_usa_results["athlete_id"].isna().sum():,}')

print(f'\n📦 BY SOURCE')
print(team_usa_results['source'].value_counts().to_string())

print(f'\n🏅 MEDALS')
print(team_usa_results['medal'].value_counts(dropna=False).to_string())

print(f'\n📅 YEAR RANGE')
for gt in ['Olympic', 'Paralympic']:
    sub = team_usa_results[team_usa_results['games_type'] == gt]
    if len(sub) > 0:
        print(f'  {gt}: {sub["games_year"].min()} — {sub["games_year"].max()} ({len(sub):,} rows)')

print(f'\n🏟️ SEASON SPLIT')
print(team_usa_results['games_season'].value_counts().to_string())

print(f'\n{"─" * 40}')
print(f'ATHLETES TABLE (post-backfill)')
print(f'{"─" * 40}')

print(f'\n📊 COUNTS')
print(f'  Total athletes:        {len(team_usa_athletes):,}')
oly = team_usa_athletes[team_usa_athletes['games_type'] == 'Olympic']
para = team_usa_athletes[team_usa_athletes['games_type'] == 'Paralympic']
print(f'  Olympic:               {len(oly):,}')
print(f'  Paralympic:            {len(para):,}')

print(f'\n📅 TEMPORAL (post-backfill)')
print(f'  Olympic:    {oly["first_games_year"].min()} — {oly["last_games_year"].max()}')
print(f'  Paralympic: {para["first_games_year"].min()} — {para["last_games_year"].max()}')

print(f'\n🏅 MEDALS (post-backfill)')
print(f'  Total medals:          {team_usa_athletes["total_medals"].sum():,.0f}')
print(f'  Olympic medalists:     {(oly["total_medals"] > 0).sum():,}')
print(f'  Paralympic medalists:  {(para["total_medals"] > 0).sum():,}')

print(f'\n⚧ GENDER (post-backfill)')
print(team_usa_athletes['gender'].value_counts(dropna=False).to_string())

print(f'\n📋 NULL CHECK (athletes)')
for col in ['name', 'gender', 'birth_date', 'games_count', 'first_games_year',
            'last_games_year', 'total_medals', 'classification_code', 'primary_sport']:
    nulls = team_usa_athletes[col].isna().sum()
    pct = nulls / len(team_usa_athletes) * 100
    flag = '⚠️' if pct > 50 else '  '
    print(f'  {flag} {col:25s} {nulls:>6,} null ({pct:.1f}%)')

print(f'\n🔗 CROSS-CHECK')
athletes_in_results = set(team_usa_results['athlete_id'].dropna())
athletes_in_table = set(team_usa_athletes['athlete_id'])
print(f'  Athletes in both:      {len(athletes_in_results & athletes_in_table):,}')
print(f'  In results only:       {len(athletes_in_results - athletes_in_table):,}')
print(f'  In table only:         {len(athletes_in_table - athletes_in_results):,} (bios without results)')

print(f'\n📋 SAMPLE PARALYMPIC (post-backfill, 5 random):')
sample_cols = ['name', 'classification_code', 'primary_sport', 'games_count',
               'total_medals', 'first_games_year', 'last_games_year', 'gender', 'source']
print(para[sample_cols].sample(5, random_state=42).to_string())

print(f'\n{"=" * 70}')
print('Paste this output back for review before proceeding to Phase 5.')
print('=' * 70)

---
## Phase 5: Enrichment — Gemini Profile Summaries + Embeddings

Generate AI-powered athlete profiles using Gemini 2.5 Flash with Google Search grounding, then create vector embeddings for similarity search.

**Two passes:**
1. **Profile summaries** — Gemini generates 2-paragraph bios from structured data + web search
2. **Vector embeddings** — Gemini Embedding model (3072 dims) encodes profiles for AlloyDB pgvector

**SDK:** google-genai (unified SDK) via Vertex AI  
**Performance:** ~50 concurrent workers with progress checkpointing and resume

In [ ]:
# ── Phase 5, Step 1: Setup + Configuration ────────────────────────
!pip install -q google-genai tqdm

import json
import time
import csv
import threading
import hashlib
from pathlib import Path
from typing import Optional, Tuple, List
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from tqdm.notebook import tqdm

class SummaryConfig:
    """Configuration for profile summary generation"""
    MODEL_NAME = 'gemini-2.5-flash'
    TEMPERATURE = 0.7
    MAX_OUTPUT_TOKENS = 1000

    BATCH_SIZE = 200
    MAX_WORKERS = 50  # Conservative for grounded calls; bump to 75 if no rate limits
    SAVE_INTERVAL = 200
    MAX_RETRIES = 3
    RETRY_DELAY = 2

    MIN_LENGTH = 100
    MAX_LENGTH = 3000

    PROGRESS_FILE = '/tmp/athlete_summaries_progress.csv'
    ERROR_LOG = '/tmp/athlete_summary_errors.csv'

class EmbeddingConfig:
    """Configuration for embedding generation"""
    MODEL_NAME = 'gemini-embedding-001'
    OUTPUT_DIMENSION = 3072

    BATCH_SIZE = 200
    MAX_WORKERS = 75  # Embeddings are lighter, can run more
    SAVE_INTERVAL = 200
    MAX_RETRIES = 3
    RETRY_DELAY = 2

    PROGRESS_FILE = '/tmp/athlete_embeddings_progress.csv'
    ERROR_LOG = '/tmp/athlete_embedding_errors.csv'

print(f"Summary config: {SummaryConfig.MODEL_NAME}, {SummaryConfig.MAX_WORKERS} workers")
print(f"Embedding config: {EmbeddingConfig.MODEL_NAME}, {EmbeddingConfig.OUTPUT_DIMENSION} dims, {EmbeddingConfig.MAX_WORKERS} workers")
print(f"Athletes to process: {len(team_usa_athletes):,}")

In [ ]:
# ── Phase 5, Step 2: Thread-local Gemini client ──────────────────
from google import genai
from google.genai import types
import google.auth

_thread_local = threading.local()

def get_client():
    """Get or create a thread-local Gemini client."""
    if getattr(_thread_local, "client", None) is None:
        credentials, _ = google.auth.default()
        _thread_local.client = genai.Client(
            vertexai=True,
            project=PROJECT_ID,
            location=REGION,
        )
    return _thread_local.client

# Test connection
try:
    test_response = get_client().models.generate_content(
        model=SummaryConfig.MODEL_NAME,
        contents='Say "API connected" and nothing else.',
        config=types.GenerateContentConfig(max_output_tokens=20),
    )
    response_text = test_response.text if test_response.text else "(empty response)"
    print(f"✅ Gemini API connected")
    print(f"   Project: {PROJECT_ID}")
    print(f"   Region: {REGION}")
    print(f"   Model: {SummaryConfig.MODEL_NAME}")
    print(f"   Response: {response_text.strip()}")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    raise

In [ ]:
# ── Phase 5, Step 3: Profile generation functions ─────────────────

def create_profile_prompt(row) -> str:
    """Build a rich prompt from available athlete data."""

    name = str(row.get('name', 'Unknown')).replace('•', ' ').strip()
    games_type = row.get('games_type', 'Olympic')

    parts = [f'Write a 2-paragraph profile for {name}, a Team USA {games_type} athlete.']

    # Structured facts
    facts = []
    if pd.notna(row.get('primary_sport')):
        facts.append(f"Sport: {row['primary_sport']}")
    if pd.notna(row.get('classification_code')):
        facts.append(f"Paralympic classification: {row['classification_code']}")
    if pd.notna(row.get('gender')):
        facts.append(f"Gender: {row['gender']}")
    if pd.notna(row.get('games_count')) and row['games_count'] > 0:
        year_str = ""
        if pd.notna(row.get('first_games_year')) and pd.notna(row.get('last_games_year')):
            fy, ly = int(row['first_games_year']), int(row['last_games_year'])
            year_str = f" ({fy})" if fy == ly else f" ({fy}–{ly})"
        facts.append(f"Games appearances: {int(row['games_count'])}{year_str}")

    medal_parts = []
    for label, col in [('Gold', 'gold_count'), ('Silver', 'silver_count'), ('Bronze', 'bronze_count')]:
        if pd.notna(row.get(col)) and row[col] > 0:
            medal_parts.append(f"{int(row[col])} {label}")
    if medal_parts:
        facts.append(f"Medals: {', '.join(medal_parts)}")

    if facts:
        parts.append("\nKnown facts:\n" + "\n".join(f"- {f}" for f in facts))

    # Bio fields (Paris 2024 Paralympic)
    bio_parts = []
    for field, label in [('reason', 'Why they compete'), ('hero', 'Hero/inspiration'),
                          ('philosophy', 'Philosophy'), ('other_sports', 'Other sports'),
                          ('coach', 'Coach'), ('hobbies', 'Hobbies'),
                          ('occupation', 'Occupation'), ('education', 'Education')]:
        if pd.notna(row.get(field)):
            bio_parts.append(f"{label}: {row[field]}")

    if bio_parts:
        parts.append("\nAdditional background:\n" + "\n".join(f"- {b}" for b in bio_parts))

    # Instructions
    instructions = """
Paragraph 1: Athletic career — key achievements, competition highlights, and what makes this athlete notable.
Paragraph 2: Personal background — how they entered their sport, their training journey, and what drives them.

Write 150-250 words total in an engaging, encyclopedic tone."""

    if pd.notna(row.get('classification_code')):
        instructions += f"\nSince this is a Paralympic athlete classified as {row['classification_code']}, briefly explain what that classification means in the context of their sport."

    parts.append(instructions)
    return "\n".join(parts)


def generate_profile(row, retries: int = SummaryConfig.MAX_RETRIES) -> Tuple[Optional[str], Optional[str]]:
    """Generate a profile summary with Google Search grounding."""

    prompt = create_profile_prompt(row)

    config = types.GenerateContentConfig(
        temperature=SummaryConfig.TEMPERATURE,
        max_output_tokens=SummaryConfig.MAX_OUTPUT_TOKENS,
        tools=[types.Tool(google_search=types.GoogleSearch())],
    )

    for attempt in range(retries):
        try:
            response = get_client().models.generate_content(
                model=SummaryConfig.MODEL_NAME,
                contents=prompt,
                config=config,
            )

            summary = None
            if hasattr(response, 'text') and response.text:
                summary = response.text
            elif hasattr(response, 'candidates') and response.candidates:
                candidate = response.candidates[0]
                if hasattr(candidate, 'content') and hasattr(candidate.content, 'parts'):
                    parts = candidate.content.parts
                    if parts and hasattr(parts[0], 'text'):
                        summary = parts[0].text

            if not summary:
                if attempt < retries - 1:
                    time.sleep(SummaryConfig.RETRY_DELAY)
                    continue
                return None, "No text in API response"

            summary = summary.strip()

            if len(summary) < SummaryConfig.MIN_LENGTH:
                if attempt < retries - 1:
                    time.sleep(SummaryConfig.RETRY_DELAY)
                    continue
                return None, f"Too short ({len(summary)} chars)"

            if len(summary) > SummaryConfig.MAX_LENGTH:
                summary = summary[:SummaryConfig.MAX_LENGTH] + "..."

            # Clean for CSV
            summary = re.sub(r'\s+', ' ', summary).strip()

            return summary, None

        except Exception as e:
            error_msg = f"{type(e).__name__}: {str(e)}"
            if attempt < retries - 1:
                time.sleep(SummaryConfig.RETRY_DELAY * (attempt + 1))
                continue
            return None, error_msg

    return None, "Max retries exceeded"


def log_error(athlete_id, name, error_msg, filename):
    """Log errors to CSV."""
    file_exists = Path(filename).exists()
    with open(filename, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['timestamp', 'athlete_id', 'name', 'error'])
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            'timestamp': datetime.now().isoformat(),
            'athlete_id': athlete_id,
            'name': name,
            'error': error_msg
        })

print("✅ Profile generation functions defined")

In [ ]:
# ── Phase 5, Step 4: Test with a single athlete ──────────────────

# Pick one well-known athlete and one sparse one
test_athletes = [
    team_usa_athletes[team_usa_athletes['name'].str.contains('Simone', na=False)].head(1),
    team_usa_athletes[team_usa_athletes['source'] == 'katiepress'].sample(1, random_state=42),
    team_usa_athletes[(team_usa_athletes['games_type'] == 'Paralympic') &
                       (team_usa_athletes['classification_code'].notna())].sample(1, random_state=42)
]

for test_df in test_athletes:
    if len(test_df) == 0:
        continue
    row = test_df.iloc[0]
    print(f"\n{'=' * 60}")
    print(f"Testing: {row['name']} ({row['games_type']}, {row.get('primary_sport', 'N/A')})")
    print(f"  Classification: {row.get('classification_code', 'N/A')}")
    print(f"  Medals: {row.get('total_medals', 0)}, Games: {row.get('games_count', 'N/A')}")
    print(f"  Source: {row.get('source', 'N/A')}")
    print(f"{'=' * 60}")

    print(f"\nPrompt preview (first 300 chars):")
    prompt = create_profile_prompt(row)
    print(prompt[:300] + "...")

    print(f"\nGenerating profile...")
    summary, error = generate_profile(row)

    if summary:
        print(f"\n✅ Success ({len(summary)} chars):")
        print(summary)
    else:
        print(f"\n❌ Error: {error}")

In [ ]:
# ── Phase 5, Step 5: Parallel summary processing ─────────────────

def process_single_profile(idx_row) -> dict:
    """Process a single athlete for summary generation."""
    idx, row = idx_row
    athlete_id = row['athlete_id']
    name = str(row.get('name', 'Unknown')).replace('•', ' ')

    summary, error = generate_profile(row)

    if summary:
        return {'athlete_id': athlete_id, 'profile_summary': summary, 'error': None}
    else:
        log_error(athlete_id, name, error, SummaryConfig.ERROR_LOG)
        return {
            'athlete_id': athlete_id,
            'profile_summary': f"[Profile generation failed: {error}]",
            'error': error
        }


def save_summary_progress(results, filename):
    """Save summary progress to CSV."""
    df = pd.DataFrame(results)[['athlete_id', 'profile_summary']]
    df.to_csv(filename, index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)
    print(f"  💾 Progress: {filename} ({len(df):,} profiles)")


def generate_all_profiles(df: pd.DataFrame) -> pd.DataFrame:
    """Generate profiles for all athletes with parallel processing + resume."""

    results = []
    errors = []
    processed_ids = set()

    # Resume from progress
    progress_path = Path(SummaryConfig.PROGRESS_FILE)
    if progress_path.exists():
        print(f"\n📂 Found progress file: {SummaryConfig.PROGRESS_FILE}")
        df_progress = pd.read_csv(SummaryConfig.PROGRESS_FILE)

        successful = df_progress[~df_progress['profile_summary'].str.contains(
            r'\[Profile generation failed', na=False, regex=True)]
        processed_ids = set(successful['athlete_id'])

        for _, row in successful.iterrows():
            results.append({
                'athlete_id': row['athlete_id'],
                'profile_summary': row['profile_summary'],
                'error': None
            })

        failed_count = len(df_progress) - len(successful)
        print(f"  ✅ Loaded {len(successful):,} successful profiles")
        print(f"  🔄 Will retry {failed_count:,} failed profiles")

    # Filter to unprocessed
    df_todo = df[~df['athlete_id'].isin(processed_ids)].copy()

    if len(df_todo) == 0:
        print(f"\n✅ All profiles already generated!")
        return pd.DataFrame(results)

    print(f"\n{'=' * 60}")
    print(f"Starting PARALLEL profile generation")
    print(f"  Total athletes:      {len(df):,}")
    print(f"  Already processed:   {len(processed_ids):,}")
    print(f"  To process:          {len(df_todo):,}")
    print(f"  Workers:             {SummaryConfig.MAX_WORKERS}")
    print(f"  Model:               {SummaryConfig.MODEL_NAME} + Google Search")
    print(f"{'=' * 60}\n")

    start_time = time.time()
    rows_to_process = list(df_todo.iterrows())
    total_batches = (len(rows_to_process) + SummaryConfig.BATCH_SIZE - 1) // SummaryConfig.BATCH_SIZE

    with tqdm(total=len(rows_to_process), desc="Generating profiles") as pbar:
        for batch_num in range(total_batches):
            start_idx = batch_num * SummaryConfig.BATCH_SIZE
            end_idx = min(start_idx + SummaryConfig.BATCH_SIZE, len(rows_to_process))
            batch = rows_to_process[start_idx:end_idx]

            with ThreadPoolExecutor(max_workers=SummaryConfig.MAX_WORKERS) as executor:
                futures = {executor.submit(process_single_profile, item): item for item in batch}

                for future in as_completed(futures):
                    result = future.result()
                    results.append(result)
                    if result['error']:
                        errors.append(result['athlete_id'])
                    pbar.update(1)

            # Checkpoint
            if len(results) % SummaryConfig.SAVE_INTERVAL == 0 or end_idx == len(rows_to_process):
                save_summary_progress(results, SummaryConfig.PROGRESS_FILE)

    # Final save
    save_summary_progress(results, SummaryConfig.PROGRESS_FILE)

    elapsed = time.time() - start_time
    print(f"\n{'=' * 60}")
    print(f"Profile Generation Complete!")
    print(f"{'=' * 60}")
    print(f"  Newly processed: {len(df_todo):,}")
    print(f"  Total profiles:  {len(results):,}")
    print(f"  Successful:      {len(results) - len(errors):,}")
    print(f"  Errors:          {len(errors):,}")
    if len(df_todo) > 0:
        print(f"  Time:            {elapsed/60:.1f} minutes")
        print(f"  Throughput:      {len(df_todo)/elapsed*60:.1f} athletes/minute")
    if errors:
        print(f"  Error rate:      {len(errors)/len(results)*100:.1f}%")

    return pd.DataFrame(results)

print("✅ Parallel summary processing functions defined")

In [ ]:
# ── Phase 5, Step 6: Run profile generation ───────────────────────
df_profiles = generate_all_profiles(team_usa_athletes)

In [ ]:
# ── Phase 5, Step 7: Summary QC ──────────────────────────────────
print('=' * 60)
print('PHASE 5a QC: PROFILE SUMMARIES')
print('=' * 60)

successful = df_profiles[~df_profiles['profile_summary'].str.contains(
    r'\[Profile generation failed', na=False, regex=True)]
failed = df_profiles[df_profiles['profile_summary'].str.contains(
    r'\[Profile generation failed', na=False, regex=True)]

print(f'\n📊 COUNTS')
print(f'  Total profiles:     {len(df_profiles):,}')
print(f'  Successful:         {len(successful):,}')
print(f'  Failed:             {len(failed):,}')
if len(failed) > 0:
    print(f'  Error rate:         {len(failed)/len(df_profiles)*100:.1f}%')

if len(successful) > 0:
    lengths = successful['profile_summary'].str.len()
    print(f'\n📏 LENGTH STATS')
    print(f'  Min:    {lengths.min():,} chars')
    print(f'  Max:    {lengths.max():,} chars')
    print(f'  Mean:   {lengths.mean():,.0f} chars')
    print(f'  Median: {lengths.median():,.0f} chars')

# Check coverage by source
merged_check = df_profiles.merge(
    team_usa_athletes[['athlete_id', 'games_type', 'source']], on='athlete_id', how='left')
print(f'\n📦 BY SOURCE')
for src in merged_check['source'].unique():
    sub = merged_check[merged_check['source'] == src]
    success = (~sub['profile_summary'].str.contains(
        r'\[Profile generation failed', na=False, regex=True)).sum()
    print(f'  {src:15s} {success:>6,} / {len(sub):>6,} successful')

print(f'\n📦 BY TYPE')
for gt in ['Olympic', 'Paralympic']:
    sub = merged_check[merged_check['games_type'] == gt]
    success = (~sub['profile_summary'].str.contains(
        r'\[Profile generation failed', na=False, regex=True)).sum()
    print(f'  {gt:15s} {success:>6,} / {len(sub):>6,} successful')

# Sample profiles
print(f'\n📋 SAMPLE PROFILES (1 Olympic, 1 Paralympic):')
for gt in ['Olympic', 'Paralympic']:
    sub = merged_check[(merged_check['games_type'] == gt) &
                        (~merged_check['profile_summary'].str.contains(
                            r'\[Profile generation failed', na=False, regex=True))]
    if len(sub) > 0:
        sample = sub.sample(1, random_state=42).iloc[0]
        print(f'\n  --- {gt} ({sample["source"]}) ---')
        print(f'  {sample["profile_summary"][:400]}...')

if len(failed) > 0:
    print(f'\n⚠️ SAMPLE FAILURES:')
    print(failed[['athlete_id', 'profile_summary']].head(5).to_string())

print(f'\n{"=" * 60}')
print('Paste this output back for review before running embeddings.')
print('=' * 60)

### Phase 5b: Vector Embeddings

Generate 3072-dimensional embeddings from structured athlete data + generated profile summaries. These power the AlloyDB pgvector similarity search in the lab.

In [ ]:
# ── Phase 5, Step 8: Embedding functions ──────────────────────────

def create_embedding_text(row, profile_summary: str) -> str:
    """Create rich text for embedding from structured data + profile."""

    name = str(row.get('name', 'Unknown')).replace('•', ' ')
    parts = [f"Team USA {row.get('games_type', '')} athlete: {name}"]

    if pd.notna(row.get('primary_sport')):
        parts.append(f"Sport: {row['primary_sport']}")
    if pd.notna(row.get('classification_code')):
        parts.append(f"Classification: {row['classification_code']}")
    if pd.notna(row.get('games_count')) and row['games_count'] > 0:
        year_str = ""
        if pd.notna(row.get('first_games_year')) and pd.notna(row.get('last_games_year')):
            fy, ly = int(row['first_games_year']), int(row['last_games_year'])
            year_str = f" ({fy}-{ly})"
        medal_str = ""
        if pd.notna(row.get('total_medals')) and row['total_medals'] > 0:
            medal_str = f", {int(row['total_medals'])} medals"
        parts.append(f"Career: {int(row['games_count'])} Games{year_str}{medal_str}")

    # Add profile summary if it's real (not an error message)
    if profile_summary and not profile_summary.startswith('[Profile generation failed'):
        parts.append(f"Profile: {profile_summary}")

    return ". ".join(parts)


def generate_embedding(text: str, retries: int = EmbeddingConfig.MAX_RETRIES) -> Tuple[Optional[List[float]], Optional[str]]:
    """Generate embedding for text."""

    for attempt in range(retries):
        try:
            response = get_client().models.embed_content(
                model=EmbeddingConfig.MODEL_NAME,
                contents=text,
                config=types.EmbedContentConfig(
                    output_dimensionality=EmbeddingConfig.OUTPUT_DIMENSION
                )
            )

            if hasattr(response, 'embeddings') and response.embeddings:
                embedding = response.embeddings[0]
                if hasattr(embedding, 'values') and embedding.values:
                    return list(embedding.values), None

            if attempt < retries - 1:
                time.sleep(EmbeddingConfig.RETRY_DELAY)
                continue
            return None, "No embedding values in response"

        except Exception as e:
            if attempt < retries - 1:
                time.sleep(EmbeddingConfig.RETRY_DELAY * (attempt + 1))
                continue
            return None, f"{type(e).__name__}: {str(e)}"

    return None, "Max retries exceeded"


def process_single_embedding(item) -> dict:
    """Process a single athlete for embedding."""
    athlete_id, row, profile_summary = item

    text = create_embedding_text(row, profile_summary)
    embedding, error = generate_embedding(text)

    if embedding:
        return {'athlete_id': athlete_id, 'embedding': embedding, 'error': None}
    else:
        name = str(row.get('name', 'Unknown')).replace('•', ' ')
        log_error(athlete_id, name, error, EmbeddingConfig.ERROR_LOG)
        return {'athlete_id': athlete_id, 'embedding': None, 'error': error}


def save_embedding_progress(results, filename):
    """Save embedding progress to CSV."""
    df = pd.DataFrame([r for r in results if r['embedding'] is not None])
    if len(df) > 0:
        df['embedding'] = df['embedding'].apply(json.dumps)
        df[['athlete_id', 'embedding']].to_csv(filename, index=False, encoding='utf-8')
    print(f"  💾 Progress: {filename} ({len(df):,} embeddings)")

print("✅ Embedding functions defined")

In [ ]:
# ── Phase 5, Step 9: Test single embedding ────────────────────────

test_row = team_usa_athletes.iloc[0]
test_profile = df_profiles[df_profiles['athlete_id'] == test_row['athlete_id']]
test_summary = test_profile.iloc[0]['profile_summary'] if len(test_profile) > 0 else ""

text = create_embedding_text(test_row, test_summary)
print(f"Embedding text (first 300 chars):\n{text[:300]}...\n")

print(f"Calling embedding API ({EmbeddingConfig.OUTPUT_DIMENSION} dims)...")
embedding, error = generate_embedding(text)

if embedding:
    print(f"✅ Success!")
    print(f"   Dimensions: {len(embedding)}")
    print(f"   Expected:   {EmbeddingConfig.OUTPUT_DIMENSION}")
    print(f"   First 5:    {embedding[:5]}")
    print(f"   ✓ Dimension check: {'PASS' if len(embedding) == EmbeddingConfig.OUTPUT_DIMENSION else 'FAIL'}")
else:
    print(f"❌ Error: {error}")


In [ ]:
# ── Phase 5, Step 10: Run embedding generation ───────────────────

# Build lookup: athlete_id → profile_summary
profile_lookup = dict(zip(df_profiles['athlete_id'], df_profiles['profile_summary']))

def generate_all_embeddings(df: pd.DataFrame) -> pd.DataFrame:
    """Generate embeddings for all athletes with parallel processing + resume."""

    results = []
    errors = []
    processed_ids = set()

    # Resume from progress
    progress_path = Path(EmbeddingConfig.PROGRESS_FILE)
    if progress_path.exists():
        print(f"\n📂 Found progress file: {EmbeddingConfig.PROGRESS_FILE}")
        df_progress = pd.read_csv(EmbeddingConfig.PROGRESS_FILE)
        df_progress['embedding'] = df_progress['embedding'].apply(
            lambda x: json.loads(x) if isinstance(x, str) and x.startswith('[') else None)
        successful = df_progress[df_progress['embedding'].notna()]
        processed_ids = set(successful['athlete_id'])

        for _, row in successful.iterrows():
            results.append({
                'athlete_id': row['athlete_id'],
                'embedding': row['embedding'],
                'error': None
            })
        print(f"  ✅ Loaded {len(successful):,} embeddings")

    # Filter to unprocessed
    df_todo = df[~df['athlete_id'].isin(processed_ids)].copy()

    if len(df_todo) == 0:
        print(f"\n✅ All embeddings already generated!")
        return pd.DataFrame(results)

    print(f"\n{'=' * 60}")
    print(f"Starting PARALLEL embedding generation")
    print(f"  Total athletes:      {len(df):,}")
    print(f"  Already processed:   {len(processed_ids):,}")
    print(f"  To process:          {len(df_todo):,}")
    print(f"  Workers:             {EmbeddingConfig.MAX_WORKERS}")
    print(f"  Model:               {EmbeddingConfig.MODEL_NAME}")
    print(f"  Dimensions:          {EmbeddingConfig.OUTPUT_DIMENSION}")
    print(f"{'=' * 60}\n")

    start_time = time.time()

    # Build items: (athlete_id, row, profile_summary)
    items = []
    for idx, row in df_todo.iterrows():
        profile = profile_lookup.get(row['athlete_id'], '')
        items.append((row['athlete_id'], row, profile))

    total_batches = (len(items) + EmbeddingConfig.BATCH_SIZE - 1) // EmbeddingConfig.BATCH_SIZE

    with tqdm(total=len(items), desc="Generating embeddings") as pbar:
        for batch_num in range(total_batches):
            start_idx = batch_num * EmbeddingConfig.BATCH_SIZE
            end_idx = min(start_idx + EmbeddingConfig.BATCH_SIZE, len(items))
            batch = items[start_idx:end_idx]

            with ThreadPoolExecutor(max_workers=EmbeddingConfig.MAX_WORKERS) as executor:
                futures = {executor.submit(process_single_embedding, item): item for item in batch}

                for future in as_completed(futures):
                    result = future.result()
                    results.append(result)
                    if result['error']:
                        errors.append(result['athlete_id'])
                    pbar.update(1)

            if len(results) % EmbeddingConfig.SAVE_INTERVAL == 0 or end_idx == len(items):
                save_embedding_progress(results, EmbeddingConfig.PROGRESS_FILE)

    save_embedding_progress(results, EmbeddingConfig.PROGRESS_FILE)

    elapsed = time.time() - start_time
    print(f"\n{'=' * 60}")
    print(f"Embedding Generation Complete!")
    print(f"{'=' * 60}")
    print(f"  Newly processed: {len(df_todo):,}")
    print(f"  Total embeddings: {len([r for r in results if r['embedding']]):,}")
    print(f"  Errors:           {len(errors):,}")
    if len(df_todo) > 0:
        print(f"  Time:             {elapsed/60:.1f} minutes")
        print(f"  Throughput:       {len(df_todo)/elapsed*60:.1f} athletes/minute")

    return pd.DataFrame(results)

# Run it
df_embeddings = generate_all_embeddings(team_usa_athletes)

In [ ]:
# ── Phase 5, Step 11: Merge enrichments into athletes table ──────

# Add profile summaries
profile_map = dict(zip(df_profiles['athlete_id'], df_profiles['profile_summary']))
team_usa_athletes['profile_summary'] = team_usa_athletes['athlete_id'].map(profile_map)

# Add embeddings
embedding_map = dict(zip(
    df_embeddings[df_embeddings['embedding'].notna()]['athlete_id'],
    df_embeddings[df_embeddings['embedding'].notna()]['embedding']
))
team_usa_athletes['embedding'] = team_usa_athletes['athlete_id'].map(embedding_map)

print(f"Enrichment merged:")
print(f"  Profile summaries: {team_usa_athletes['profile_summary'].notna().sum():,} / {len(team_usa_athletes):,}")
print(f"  Embeddings:        {team_usa_athletes['embedding'].notna().sum():,} / {len(team_usa_athletes):,}")

# Save enriched athletes (embeddings as JSON)
enriched = team_usa_athletes.copy()
enriched['embedding'] = enriched['embedding'].apply(
    lambda x: json.dumps(x) if isinstance(x, list) else None)

ath_path = '/tmp/team_usa_athletes_enriched.csv'
enriched.to_csv(ath_path, index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)
!gsutil cp {ath_path} gs://class-demo/team-usa/enriched/team_usa_athletes_enriched.csv

# Also save results table (unchanged but checkpoint alongside)
res_path = '/tmp/team_usa_results.csv'
team_usa_results.to_csv(res_path, index=False)
!gsutil cp {res_path} gs://class-demo/team-usa/enriched/team_usa_results.csv

print(f"\n✅ Enriched checkpoints saved to gs://class-demo/team-usa/enriched/")

In [ ]:
# ── PHASE 5 QC REPORT ─────────────────────────────────────────────
print('=' * 70)
print('PHASE 5 QC: ENRICHED ATHLETES')
print('=' * 70)

print(f'\n📊 PROFILE SUMMARIES')
has_profile = team_usa_athletes['profile_summary'].notna()
has_real_profile = has_profile & ~team_usa_athletes['profile_summary'].str.contains(
    r'\[Profile generation failed', na=False, regex=True)
print(f'  With profile:        {has_real_profile.sum():,} / {len(team_usa_athletes):,} ({has_real_profile.sum()/len(team_usa_athletes)*100:.1f}%)')
print(f'  Failed:              {(has_profile & ~has_real_profile).sum():,}')

if has_real_profile.sum() > 0:
    lengths = team_usa_athletes.loc[has_real_profile, 'profile_summary'].str.len()
    print(f'  Length — min: {lengths.min():,}, max: {lengths.max():,}, median: {lengths.median():,.0f}')

print(f'\n📊 EMBEDDINGS')
has_emb = team_usa_athletes['embedding'].notna()
print(f'  With embedding:      {has_emb.sum():,} / {len(team_usa_athletes):,} ({has_emb.sum()/len(team_usa_athletes)*100:.1f}%)')
if has_emb.sum() > 0:
    sample_emb = team_usa_athletes.loc[has_emb, 'embedding'].iloc[0]
    dim = len(sample_emb) if isinstance(sample_emb, list) else 'N/A'
    print(f'  Dimensions:          {dim}')
    print(f'  Expected:            {EmbeddingConfig.OUTPUT_DIMENSION}')

print(f'\n📦 COVERAGE BY TYPE')
for gt in ['Olympic', 'Paralympic']:
    sub = team_usa_athletes[team_usa_athletes['games_type'] == gt]
    prof = (sub['profile_summary'].notna() & ~sub['profile_summary'].str.contains(
        r'\[Profile generation failed', na=False, regex=True)).sum()
    emb = sub['embedding'].notna().sum()
    print(f'  {gt:15s} profiles: {prof:>6,}/{len(sub):>6,}  embeddings: {emb:>6,}/{len(sub):>6,}')

print(f'\n📦 COVERAGE BY SOURCE')
for src in team_usa_athletes['source'].unique():
    sub = team_usa_athletes[team_usa_athletes['source'] == src]
    prof = (sub['profile_summary'].notna() & ~sub['profile_summary'].str.contains(
        r'\[Profile generation failed', na=False, regex=True)).sum()
    emb = sub['embedding'].notna().sum()
    print(f'  {src:15s} profiles: {prof:>6,}/{len(sub):>6,}  embeddings: {emb:>6,}/{len(sub):>6,}')

print(f'\n📋 SAMPLE ENRICHED ATHLETES:')
for gt in ['Olympic', 'Paralympic']:
    sub = team_usa_athletes[(team_usa_athletes['games_type'] == gt) & has_real_profile]
    if len(sub) > 0:
        s = sub.sample(1, random_state=99).iloc[0]
        print(f'\n  --- {gt}: {s["name"]} ({s.get("primary_sport", "N/A")}) ---')
        print(f'  Classification: {s.get("classification_code", "N/A")}')
        print(f'  Profile: {str(s["profile_summary"])[:300]}...')
        emb = s.get('embedding')
        if isinstance(emb, list):
            print(f'  Embedding: [{emb[0]:.6f}, {emb[1]:.6f}, ... ] ({len(emb)} dims)')

print(f'\n📊 FINAL TABLE SHAPE')
print(f'  Athletes: {len(team_usa_athletes):,} rows × {len(team_usa_athletes.columns)} columns')
print(f'  Results:  {len(team_usa_results):,} rows × {len(team_usa_results.columns)} columns')
print(f'  Columns:  {list(team_usa_athletes.columns)}')

print(f'\n✅ FILES ON GCS:')
print(f'  gs://class-demo/team-usa/enriched/team_usa_athletes_enriched.csv')
print(f'  gs://class-demo/team-usa/enriched/team_usa_results.csv')

print(f'\n{"=" * 70}')
print('Paste this output back for review before proceeding to Phase 6.')
print('=' * 70)

---
## Phase 6: Final Validation & Data Card

Sanity queries that preview what students will run in BigQuery, a vector similarity smoke test, and the dataset's official data card.

In [ ]:
# ── Phase 6, Step 1: Dataset overview queries ─────────────────────────
import numpy as np

print('=' * 70)
print('DATASET OVERVIEW')
print('=' * 70)

# Temporal span
print('\n📅 TEMPORAL COVERAGE')
for gt in ['Olympic', 'Paralympic']:
    sub = team_usa_athletes[team_usa_athletes['games_type'] == gt]
    first = sub['first_games_year'].dropna()
    last = sub['last_games_year'].dropna()
    if len(first) > 0:
        print(f'  {gt:15s} {int(first.min())}–{int(last.max())}')

# Gender breakdown
print('\n👥 GENDER DISTRIBUTION')
gender_by_type = team_usa_athletes.groupby(['games_type', 'gender']).size().unstack(fill_value=0)
print(gender_by_type.to_string())

# Sport diversity
print('\n🏅 SPORT DIVERSITY')
for gt in ['Olympic', 'Paralympic']:
    sub = team_usa_athletes[team_usa_athletes['games_type'] == gt]
    n_sports = sub['primary_sport'].nunique()
    top_3 = sub['primary_sport'].value_counts().head(3)
    print(f'  {gt}: {n_sports} sports — top 3: {", ".join(f"{s} ({c})" for s, c in top_3.items())}')

# Medal leaders
print('\n🥇 TOP 10 MEDALISTS (ALL-TIME)')
top = team_usa_athletes.nlargest(10, 'total_medals')[
    ['name', 'games_type', 'primary_sport', 'gold_count', 'total_medals', 'games_count']
].reset_index(drop=True)
top.index = top.index + 1
print(top.to_string())

# Multi-Games athletes
print('\n🔁 MULTI-GAMES ATHLETES')
for gt in ['Olympic', 'Paralympic']:
    sub = team_usa_athletes[(team_usa_athletes['games_type'] == gt) & (team_usa_athletes['games_count'] >= 2)]
    multi4 = (team_usa_athletes['games_type'] == gt) & (team_usa_athletes['games_count'] >= 4)
    print(f'  {gt:15s} 2+ Games: {len(sub):,}   4+ Games: {multi4.sum():,}')

In [ ]:
# ── Phase 6, Step 2: Paralympic classification analysis ────────────────

print('=' * 70)
print('PARALYMPIC CLASSIFICATION ANALYSIS')
print('=' * 70)

para = team_usa_athletes[team_usa_athletes['games_type'] == 'Paralympic']

print(f'\n📊 COVERAGE')
has_class = para['classification_code'].notna()
print(f'  Athletes with classification: {has_class.sum():,} / {len(para):,} ({has_class.sum()/len(para)*100:.1f}%)')

# Top classification codes
print(f'\n📋 TOP 15 CLASSIFICATION CODES')
class_counts = para['classification_code'].value_counts().head(15)
for code, count in class_counts.items():
    print(f'  {str(code):20s} {count:>4} athletes')

# Classification by sport
print(f'\n🏊 CLASSIFICATION CODES BY SPORT (top 5 sports)')
para_with_class = para[has_class]
top_sports = para_with_class['primary_sport'].value_counts().head(5)
for sport in top_sports.index:
    sub = para_with_class[para_with_class['primary_sport'] == sport]
    codes = sub['classification_code'].value_counts().head(5)
    code_str = ', '.join(f'{c}({n})' for c, n in codes.items())
    print(f'  {sport:20s} [{len(sub)} athletes] {code_str}')

# Classification gap analysis
print(f'\n⚠️ MISSING CLASSIFICATION')
missing_class = para[~has_class]
by_source = missing_class['source'].value_counts()
for src, count in by_source.items():
    print(f'  {src:15s} {count:>4} athletes without classification')
decades = missing_class['first_games_year'].dropna().apply(lambda y: f"{int(y)//10*10}s")
if len(decades) > 0:
    print(f'  Decades: {decades.value_counts().sort_index().to_dict()}')

In [ ]:
# ── Phase 6, Step 3: Vector similarity smoke test ─────────────────────
from numpy.linalg import norm

def cosine_similarity(a, b):
    """Compute cosine similarity between two vectors."""
    a, b = np.array(a), np.array(b)
    return np.dot(a, b) / (norm(a) * norm(b))

# Get athletes with embeddings
has_emb = team_usa_athletes[team_usa_athletes['embedding'].apply(lambda x: isinstance(x, list))]

print('=' * 70)
print('VECTOR SIMILARITY SMOKE TEST')
print('=' * 70)
print(f'Athletes with valid embeddings: {len(has_emb):,}\n')

# Test 1: Find athletes similar to a well-known swimmer
query_name = 'Swimming'
swimmers = has_emb[has_emb['primary_sport'].str.contains(query_name, na=False, case=False)]
if len(swimmers) >= 2:
    # Pick the swimmer with the most medals as anchor
    anchor = swimmers.nlargest(1, 'total_medals').iloc[0]
    anchor_emb = anchor['embedding']

    print(f'🏊 TEST 1: Athletes similar to {anchor["name"]} ({anchor["games_type"]}, {int(anchor.get("total_medals", 0))} medals)')

    # Compare against all athletes
    sims = []
    for _, row in has_emb.iterrows():
        if row['athlete_id'] == anchor['athlete_id']:
            continue
        sim = cosine_similarity(anchor_emb, row['embedding'])
        sims.append({
            'name': row['name'], 'sport': row.get('primary_sport', 'N/A'),
            'type': row['games_type'], 'medals': int(row.get('total_medals', 0) if pd.notna(row.get('total_medals')) else 0),
            'classification': row.get('classification_code', None),
            'similarity': sim
        })

    sims_df = pd.DataFrame(sims).nlargest(10, 'similarity')
    print(f'\n  Top 10 most similar:')
    for i, (_, r) in enumerate(sims_df.iterrows(), 1):
        cls = f' [{r["classification"]}]' if pd.notna(r.get('classification')) else ''
        print(f'  {i:>2}. {r["similarity"]:.4f}  {r["name"]:30s} {r["sport"]:20s} {r["type"]}{cls}  ({r["medals"]} medals)')

print()

# Test 2: Cross-type similarity — find Paralympic athletes similar to an Olympic athlete
print(f'🤝 TEST 2: Paralympic athletes most similar to {anchor["name"]}')
para_sims = [s for s in sims if s['type'] == 'Paralympic']
para_top = sorted(para_sims, key=lambda x: x['similarity'], reverse=True)[:5]
for i, r in enumerate(para_top, 1):
    cls = f' [{r["classification"]}]' if pd.notna(r.get('classification')) else ''
    print(f'  {i}. {r["similarity"]:.4f}  {r["name"]:30s} {r["sport"]:20s}{cls}  ({r["medals"]} medals)')

print()

# Test 3: Sanity check — dissimilar sports should have lower similarity
print(f'🎯 TEST 3: Similarity sanity check (same sport vs different)')
random_other = has_emb[~has_emb['primary_sport'].str.contains(query_name, na=False, case=False)].sample(50, random_state=42)
same_sport_sims = [s['similarity'] for s in sims if query_name.lower() in str(s.get('sport', '')).lower()][:50]
diff_sport_sims = [cosine_similarity(anchor_emb, row['embedding']) for _, row in random_other.iterrows()]

print(f'  Same sport avg similarity:  {np.mean(same_sport_sims):.4f} (n={len(same_sport_sims)})')
print(f'  Diff sport avg similarity:  {np.mean(diff_sport_sims):.4f} (n={len(diff_sport_sims)})')
print(f'  Delta:                      {np.mean(same_sport_sims) - np.mean(diff_sport_sims):.4f}')
print(f'  ✓ Embeddings discriminate by sport: {"PASS ✅" if np.mean(same_sport_sims) > np.mean(diff_sport_sims) else "FAIL ❌"}')

In [ ]:
# ── Phase 6, Step 4: Column cleanup + schema freeze ──────────────────

# Define final column order for downstream consumers
ATHLETE_COLUMNS = [
    # Identity
    'athlete_id', 'name', 'gender', 'birth_date', 'birth_place', 'birth_country',
    # Physical
    'height_cm', 'weight_kg',
    # Competition
    'games_type', 'primary_sport', 'classification_code',
    # Career stats
    'first_games_year', 'last_games_year', 'games_count',
    'gold_count', 'silver_count', 'bronze_count', 'total_medals',
    # Bio fields
    'reason', 'hero', 'philosophy', 'other_sports', 'coach',
    'hobbies', 'occupation', 'education',
    # Enrichment
    'profile_summary', 'embedding',
    # Metadata
    'source',
]

RESULT_COLUMNS = [
    'athlete_id', 'athlete_name', 'games_year', 'games_season',
    'games_type', 'sport', 'discipline', 'event', 'medal',
    'classification_code',
]

# Apply column order (drop any extras, keep only what's defined)
athletes_final = team_usa_athletes[[c for c in ATHLETE_COLUMNS if c in team_usa_athletes.columns]].copy()
results_final = team_usa_results[[c for c in RESULT_COLUMNS if c in team_usa_results.columns]].copy()

# Check for missing expected columns
missing_ath = [c for c in ATHLETE_COLUMNS if c not in team_usa_athletes.columns]
missing_res = [c for c in RESULT_COLUMNS if c not in team_usa_results.columns]
if missing_ath:
    print(f'⚠️ Athletes missing columns: {missing_ath}')
if missing_res:
    print(f'⚠️ Results missing columns: {missing_res}')

print(f'Athletes schema: {len(athletes_final.columns)} columns')
print(f'Results schema:  {len(results_final.columns)} columns')
print(f'\nAthletes columns:\n  {list(athletes_final.columns)}')
print(f'\nResults columns:\n  {list(results_final.columns)}')

In [ ]:
# ── Phase 6, Step 5: Dataset Data Card ───────────────────────────────

print('╔' + '═' * 68 + '╗')
print('║' + ' TEAM USA OLYMPIC & PARALYMPIC ATHLETES — DATA CARD'.center(68) + '║')
print('╚' + '═' * 68 + '╝')

print(f"""
📋 DATASET SUMMARY
  Name:               Team USA Olympic & Paralympic Athletes
  Version:            1.0
  Created:            {datetime.now().strftime('%Y-%m-%d')}
  Pipeline:           Colab Enterprise notebook (6 phases)

📊 TABLES
  Athletes:           {len(athletes_final):,} rows × {len(athletes_final.columns)} columns
  Results:            {len(results_final):,} rows × {len(results_final.columns)} columns

👥 ATHLETE COVERAGE
  Olympic:            {len(athletes_final[athletes_final['games_type']=='Olympic']):,} athletes ({athletes_final[athletes_final['games_type']=='Olympic']['first_games_year'].dropna().min():.0f}–{athletes_final[athletes_final['games_type']=='Olympic']['last_games_year'].dropna().max():.0f})
  Paralympic:         {len(athletes_final[athletes_final['games_type']=='Paralympic']):,} athletes ({athletes_final[athletes_final['games_type']=='Paralympic']['first_games_year'].dropna().min():.0f}–{athletes_final[athletes_final['games_type']=='Paralympic']['last_games_year'].dropna().max():.0f})
""")

# Compute stats
oly = athletes_final[athletes_final['games_type'] == 'Olympic']
para = athletes_final[athletes_final['games_type'] == 'Paralympic']

has_height = athletes_final['height_cm'].notna().sum()
has_weight = athletes_final['weight_kg'].notna().sum()
has_class = athletes_final['classification_code'].notna().sum()
has_profile = athletes_final['profile_summary'].notna().sum()
has_emb = athletes_final['embedding'].apply(lambda x: isinstance(x, list)).sum()
total_medals = athletes_final['total_medals'].sum()
medalists = (athletes_final['total_medals'] > 0).sum()
n_sports = athletes_final['primary_sport'].nunique()

print(f"""📈 FEATURE COVERAGE
  Height:             {has_height:,} / {len(athletes_final):,} ({has_height/len(athletes_final)*100:.0f}%) — Olympic only
  Weight:             {has_weight:,} / {len(athletes_final):,} ({has_weight/len(athletes_final)*100:.0f}%) — Olympic only
  Classification:     {has_class:,} / {len(para):,} Paralympic ({has_class/len(para)*100:.0f}%)
  Profile summary:    {has_profile:,} / {len(athletes_final):,} ({has_profile/len(athletes_final)*100:.0f}%)
  Embedding (3072d):  {has_emb:,} / {len(athletes_final):,} ({has_emb/len(athletes_final)*100:.0f}%)

🏅 COMPETITION STATS
  Total medals:       {int(total_medals):,} across {medalists:,} medalists
  Sports:             {n_sports} unique
  Results records:    {len(results_final):,} (individual event participations/medals)

📦 DATA SOURCES
  keithgalli          Olympic bios + results (1896–2022)
  paris2024           Olympic + Paralympic athletes (2024)
  piterfm             Paralympic Tokyo 2020 + Paris 2024
  katiepress          Paralympic medal records (1960–2018)

🤖 AI ENRICHMENT
  Model (summaries):  Gemini 2.5 Flash + Google Search grounding
  Model (embeddings): gemini-embedding-001 (3072 dimensions)
  Profiles:           2-paragraph athlete bios, 150–250 words

📁 FILES ON GCS
  gs://class-demo/team-usa/final/team_usa_athletes.csv
  gs://class-demo/team-usa/final/team_usa_results.csv

🎯 INTENDED USE
  Lab: "Build an AI Sports Analytics Agent with Team USA"
  BigQuery:     Clustering, BQML, analytical queries
  AlloyDB:      pgvector similarity search (3072-dim)
  ADK Agent:    Conversational sports analytics with tool use

⚠️ KNOWN LIMITATIONS
  - Paralympic physical attributes (height/weight) effectively zero
  - Classification coverage 47% for Paralympic (pre-1990s gap)
  - Katiepress names abbreviated (LAST I format), no cross-matching
  - 119 athletes (~1%) with unknown gender
  - Beijing 2022 Winter Paralympics not covered (data gap)
""")

print('═' * 70)
print('NOTEBOOK COMPLETE — Ready for BigQuery + AlloyDB load')
print('═' * 70)

---
## Phase 6.5: Name Cleanup

Fix three issues before students see the data:
1. **Bullet separators** (`•`) → spaces (keithgalli source, ~10K athletes)
2. **ALL CAPS abbreviated** (`ZORN T`) → full proper names via Gemini extraction from existing profiles (katiepress, ~1,165 athletes)
3. **Inconsistent casing** → standardized title case across all sources

In [ ]:
# ── Phase 6.5, Step 0: Load enriched data from GCS ────────────────────
import json

# Load the enriched athletes with profiles and embeddings
!gsutil cp gs://class-demo/team-usa/enriched/team_usa_athletes_enriched.csv /tmp/
!gsutil cp gs://class-demo/team-usa/enriched/team_usa_results.csv /tmp/

team_usa_athletes = pd.read_csv('/tmp/team_usa_athletes_enriched.csv')
team_usa_results = pd.read_csv('/tmp/team_usa_results.csv')

# Parse embeddings back from JSON strings to lists
team_usa_athletes['embedding'] = team_usa_athletes['embedding'].apply(
    lambda x: json.loads(x) if isinstance(x, str) and x.startswith('[') else None)

print(f'Loaded from GCS:')
print(f'  Athletes: {len(team_usa_athletes):,} rows')
print(f'  Results:  {len(team_usa_results):,} rows')
print(f'  Profiles: {team_usa_athletes["profile_summary"].notna().sum():,}')
print(f'  Embeddings: {team_usa_athletes["embedding"].apply(lambda x: isinstance(x, list)).sum():,}')

In [ ]:
# ── Phase 6.5, Step 1: Name cleanup utilities ─────────────────────────

def smart_title_case(name):
    """Title case with handling for Mc/O' prefixes and suffixes."""
    if not name or not isinstance(name, str):
        return name

    result = name.lower().title()

    # Fix Mc prefix: Mcdonald → McDonald, Mcconnell → McConnell
    result = re.sub(r'\bMc([a-z])', lambda m: 'Mc' + m.group(1).upper(), result)

    # Fix O' prefix (usually handled by title() but ensure it)
    result = re.sub(r"\bO'([a-z])", lambda m: "O'" + m.group(1).upper(), result)

    # Fix Roman numeral suffixes: Ii → II, Iii → III, Iv → IV
    result = re.sub(r'\bIi\b', 'II', result)
    result = re.sub(r'\bIii\b', 'III', result)
    result = re.sub(r'\bIv\b', 'IV', result)

    return result.strip()


def clean_name_mechanical(name):
    """Mechanical cleanup: bullet separators → spaces, then smart title case."""
    if not name or not isinstance(name, str):
        return name

    # Replace bullet separator with space
    name = name.replace('•', ' ')

    # Collapse multiple spaces
    name = re.sub(r'\s+', ' ', name).strip()

    # Apply smart title case
    return smart_title_case(name)


# Apply mechanical cleanup to ALL athletes first
team_usa_athletes['name_original'] = team_usa_athletes['name']
team_usa_athletes['name'] = team_usa_athletes['name'].apply(clean_name_mechanical)

# Preview
print('Mechanical cleanup applied to all 12,222 athletes\n')
print('Sample keithgalli fixes:')
kg = team_usa_athletes[team_usa_athletes['source'] == 'keithgalli'].sample(5, random_state=42)
for _, r in kg.iterrows():
    if r['name'] != r['name_original']:
        print(f'  {r["name_original"]:40s} → {r["name"]}')

print('\nSample katiepress (still abbreviated, will fix next):')
kp = team_usa_athletes[team_usa_athletes['source'] == 'katiepress'].sample(5, random_state=42)
for _, r in kp.iterrows():
    print(f'  {r["name_original"]:40s} → {r["name"]}')

In [ ]:
# ── Phase 6.5, Step 1b: Fix name order for paris2024/piterfm ──────────

def flip_name_order(name):
    """Flip 'Last First' to 'First Last' for 2-word names."""
    if not name or not isinstance(name, str):
        return name
    words = name.split()
    if len(words) == 2:
        return f"{words[1]} {words[0]}"
    return name  # Don't flip 3+ word names, too risky

# Apply to paris2024 and piterfm sources only
mask = team_usa_athletes['source'].isin(['paris2024', 'piterfm'])
team_usa_athletes.loc[mask, 'name'] = team_usa_athletes.loc[mask, 'name'].apply(flip_name_order)

# Verify
print('Name order fixed for paris2024 and piterfm:')
for src in ['paris2024', 'piterfm']:
    samples = team_usa_athletes[team_usa_athletes['source'] == src].sample(3, random_state=42)
    names = ', '.join(samples['name'].tolist())
    print(f'  {src:15s} {names}')

In [ ]:
# ── Phase 6.5, Step 2: Extract full names from profiles (katiepress) ──

NAME_PROGRESS_FILE = '/tmp/katiepress_name_extraction.csv'

def is_abbreviated(name):
    """Check if name is 'LastName I' format or single word."""
    if not name or not isinstance(name, str):
        return False
    words = name.split()
    if len(words) == 1:
        return True
    if len(words) == 2 and len(words[1]) == 1:
        return True
    return False


def extract_name(athlete_id, original_name, profile, retries=3):
    """Extract full name from profile text."""

    if not profile or profile.startswith('[Profile generation failed'):
        return None, "No profile"

    prompt = f"""Extract the athlete's full name from this profile.

Database name: "{original_name}"

Profile:
{str(profile)[:1500]}

Return ONLY the full name as it appears in the profile, or "NOT_FOUND"."""

    config = types.GenerateContentConfig(
        temperature=0.0,
        max_output_tokens=256,
    )

    for attempt in range(retries):
        try:
            response = get_client().models.generate_content(
                model=SummaryConfig.MODEL_NAME,
                contents=prompt,
                config=config,
            )

            extracted = response.text.strip() if response.text else None

            if not extracted or 'NOT_FOUND' in extracted.upper():
                return None, "Not found in profile"

            # Clean up
            extracted = extracted.strip('"\'.,')
            words = extracted.split()

            # Validate: 2+ words, no single-letter words (truncation)
            if len(words) >= 2 and all(len(w) >= 2 for w in words):
                # Verify it actually appears in profile
                profile_lower = profile.lower()
                name_lower = extracted.lower()
                if name_lower in profile_lower:
                    return smart_title_case(extracted), None
                # Try first + last only
                simple = f"{words[0]} {words[-1]}"
                if simple.lower() in profile_lower:
                    return smart_title_case(simple), None
                # Both parts appear separately
                if words[0].lower() in profile_lower and words[-1].lower() in profile_lower:
                    return smart_title_case(extracted), None
                return None, f"Not validated in profile: {extracted}"

            return None, f"Invalid format: {extracted}"

        except Exception as e:
            if '429' in str(e) or 'RESOURCE_EXHAUSTED' in str(e):
                time.sleep(5 * (attempt + 1))
                if attempt == retries - 1:
                    return None, "Rate limited"
                continue
            if attempt < retries - 1:
                time.sleep(2)
                continue
            return None, str(type(e).__name__)

    return None, "Max retries"


# Load previous progress
previous = {}
if Path(NAME_PROGRESS_FILE).exists():
    df_prev = pd.read_csv(NAME_PROGRESS_FILE)
    for _, row in df_prev.iterrows():
        if pd.notna(row.get('clean_name')) and row['clean_name'] != row['original']:
            previous[row['athlete_id']] = row['clean_name']
            team_usa_athletes.loc[team_usa_athletes['athlete_id'] == row['athlete_id'], 'name'] = row['clean_name']

print(f'Loaded {len(previous)} successful extractions from previous run')

# Find remaining abbreviated names
katiepress = team_usa_athletes[team_usa_athletes['source'] == 'katiepress']
abbreviated = katiepress[katiepress['name'].apply(is_abbreviated)].copy()

print(f'Found {len(abbreviated)} remaining abbreviated names')

if len(abbreviated) == 0:
    print('✅ All katiepress names are now complete!')
else:
    print(f'Sample: {abbreviated["name"].head(10).tolist()}\n')

    results = []
    successful_count = 0

    with tqdm(total=len(abbreviated), desc="Extracting names") as pbar:
        with ThreadPoolExecutor(max_workers=15) as executor:
            futures = {}
            for _, row in abbreviated.iterrows():
                future = executor.submit(
                    extract_name,
                    row['athlete_id'],
                    row['name_original'],
                    row.get('profile_summary', '')
                )
                futures[future] = row

            for future in as_completed(futures):
                row = futures[future]
                extracted, error = future.result()

                result = {
                    'athlete_id': row['athlete_id'],
                    'original': row['name'],
                    'clean_name': extracted if extracted else row['name'],
                    'error': error
                }
                results.append(result)

                if extracted:
                    team_usa_athletes.loc[
                        team_usa_athletes['athlete_id'] == row['athlete_id'], 'name'] = extracted
                    successful_count += 1

                pbar.update(1)
                pbar.set_postfix({'found': successful_count})

    # Save progress
    all_results = results + [{'athlete_id': aid, 'original': '', 'clean_name': name, 'error': None}
                              for aid, name in previous.items()]
    pd.DataFrame(all_results).to_csv(NAME_PROGRESS_FILE, index=False)

    # Stats
    print(f'\n{"=" * 60}')
    print(f'📊 EXTRACTION RESULTS')
    print(f'  Processed:  {len(results)}')
    print(f'  Found:      {successful_count} ({successful_count/len(results)*100:.1f}%)')
    print(f'  Remaining:  {len(results) - successful_count}')

    successes = [r for r in results if r['clean_name'] != r['original']]
    if successes:
        print(f'\n✅ SUCCESSFUL EXTRACTIONS')
        for r in successes[:20]:
            print(f'  {r["original"]:25s} → {r["clean_name"]}')

    failed = [r for r in results if r['clean_name'] == r['original']]
    if failed:
        error_types = pd.Series([r['error'] for r in failed]).value_counts()
        print(f'\n⚠️ FAILURE REASONS')
        for err, count in error_types.items():
            print(f'  {err}: {count}')
        print(f'\n  Examples:')
        for r in failed[:5]:
            print(f'  {r["original"]:25s} ({r["error"]})')

    # Final check
    still_abbrev = team_usa_athletes[
        (team_usa_athletes['source'] == 'katiepress') &
        (team_usa_athletes['name'].apply(is_abbreviated))
    ]
    print(f'\n📋 FINAL: {len(still_abbrev)} still abbreviated out of {len(katiepress)}')

In [ ]:
# ── Phase 6.5, Step 2c: Rescue names from mismatch errors ────────────

# Reload the search results
df_search = pd.read_csv(SEARCH_PROGRESS_FILE)

# Find "last name mismatch" and "invalid" entries
rescuable = df_search[df_search['error'].str.contains('Last name mismatch|Invalid', na=False)]
print(f'Attempting to rescue {len(rescuable)} names from failed extractions\n')

rescued = 0
for _, row in rescuable.iterrows():
    error = row['error']
    original = row['original']

    # Extract the name from the error message
    raw = error.replace('Last name mismatch: ', '').replace('Invalid: ', '')

    # Aggressive cleanup
    raw = re.sub(r'\[cite.*', '', raw)           # Remove citation artifacts
    raw = re.sub(r'\*\*', '', raw)               # Remove markdown bold
    raw = re.sub(r'\s*\.?\s*$', '', raw)         # Trailing periods/spaces

    # Remove common preambles
    for prefix in ['The full name of the US Paralympic athlete is ',
                   'The full name of the US Paralympic athlete ',
                   'The full name of the US Paralympic ',
                   'The full name of the US ',
                   'The US Paralympic athlete is ',
                   "The US Paralympic athlete's full name is "]:
        if raw.startswith(prefix):
            raw = raw[len(prefix):]

    # Remove suffixes like "is the full name..."
    for suffix in [' is the full name', '. While the search results',
                   "'s full name is not explicitly"]:
        idx = raw.find(suffix)
        if idx > 0:
            raw = raw[:idx]

    raw = raw.strip('"\'.,;: ')

    if not raw:
        continue

    # Take first two or three words as the name
    words = raw.split()
    if len(words) >= 2 and all(len(w) >= 2 for w in words[:2]):
        # Use first + last (skip middle names/suffixes)
        if len(words) > 3:
            name = f"{words[0]} {words[1]}"
        else:
            name = ' '.join(words)

        clean = smart_title_case(name)

        # Apply
        team_usa_athletes.loc[
            team_usa_athletes['athlete_id'] == row['athlete_id'], 'name'] = clean
        rescued += 1
        print(f'  {original:25s} -> {clean}')

print(f'\n✅ Rescued {rescued} names')

# Final count
still_abbrev = team_usa_athletes[
    (team_usa_athletes['source'] == 'katiepress') &
    (team_usa_athletes['name'].apply(is_abbreviated))
]
print(f'📋 Still abbreviated: {len(still_abbrev)} out of {len(team_usa_athletes[team_usa_athletes["source"] == "katiepress"])}')

In [ ]:
# ── Phase 6.5, Step 2c: Manual rescue from error messages ────────────

# Direct mappings where we can clearly see the correct name
manual_fixes = {
    'Lewis B': 'Brett Lewis',
    'Kelly D': 'Daniel Kelly',
    'Stein R': 'Ronald Stein',
    'Smith J': 'June Smith',
    'Burkett H': 'Harold Burkett',
    'Quinlan D': 'Darleen Quinlan',
    'Brinkmann C': 'Curt Brinkman',
    'Reed R': 'Raphew Tyrone Reed Jr',
    'Stuewevandello D': 'Donald Vandello',
    'Classonlevis R': 'Reno Levis',
    'Crasedonaldson': 'Crase Donaldson',
    'Cameron': 'James Cameron',
    'Noe M': 'Michael Noe',
    'Martin P': 'Patricia Martin',
    'Parker M': 'Martin Parker',
    'Lewellyn J': 'J. Llewellyn',  # Only initial available
}

applied = 0
for current_name, fix in manual_fixes.items():
    mask = (team_usa_athletes['source'] == 'katiepress') & (team_usa_athletes['name'] == current_name)
    matches = mask.sum()
    if matches > 0:
        team_usa_athletes.loc[mask, 'name'] = smart_title_case(fix)
        applied += 1
        print(f'  {current_name:30s} -> {smart_title_case(fix)}')
    else:
        print(f'  {current_name:30s} -> NOT FOUND in dataframe')

print(f'\n✅ Applied {applied} manual fixes')

# Final count
still_abbrev = team_usa_athletes[
    (team_usa_athletes['source'] == 'katiepress') &
    (team_usa_athletes['name'].apply(is_abbreviated))
]
print(f'📋 Still abbreviated: {len(still_abbrev)} out of {len(team_usa_athletes[team_usa_athletes["source"] == "katiepress"])}')

# Show what's left
if len(still_abbrev) > 0:
    print(f'\nRemaining abbreviated names:')
    for name in sorted(still_abbrev['name'].tolist()):
        print(f'  {name}')

In [ ]:
# ── Phase 6.5, Step 3: Update results table + QC ─────────────────────

# Sync athlete_name in results table
name_lookup = dict(zip(team_usa_athletes['athlete_id'], team_usa_athletes['name']))
team_usa_results['athlete_name'] = team_usa_results['athlete_id'].map(name_lookup).fillna(
    team_usa_results['athlete_name'].apply(clean_name_mechanical))

# Also update athletes_final and results_final if they exist
if 'athletes_final' in dir():
    athletes_final['name'] = team_usa_athletes['name']
if 'results_final' in dir():
    results_final['athlete_name'] = team_usa_results['athlete_name']

# QC Report
print('=' * 70)
print('PHASE 6.5 QC: NAME CLEANUP')
print('=' * 70)

# Check for remaining bullets
bullets_remaining = team_usa_athletes['name'].str.contains('•', na=False).sum()
print(f'\n🔍 REMAINING ISSUES')
print(f'  Bullet chars remaining:   {bullets_remaining}')

# Check for all-caps names (3+ consecutive uppercase letters suggest uncleaned)
all_caps = team_usa_athletes['name'].apply(
    lambda x: bool(re.search(r'[A-Z]{3,}', str(x))) if pd.notna(x) else False).sum()
print(f'  All-caps patterns:        {all_caps}')

# Casing distribution
has_upper_lower = team_usa_athletes['name'].apply(
    lambda x: bool(re.search(r'[a-z]', str(x)) and re.search(r'[A-Z]', str(x)))
    if pd.notna(x) else False).sum()
print(f'  Mixed case (normal):      {has_upper_lower:,} / {len(team_usa_athletes):,}')

# Katiepress extraction stats
kp_athletes = team_usa_athletes[team_usa_athletes['source'] == 'katiepress']
kp_multiword = kp_athletes['name'].apply(lambda x: len(str(x).split()) >= 2).sum()
print(f'\n📋 KATIEPRESS NAME RECOVERY')
kp_athletes = team_usa_athletes[team_usa_athletes['source'] == 'katiepress']
still_abbrev = kp_athletes['name'].apply(is_abbreviated).sum()
print(f'  Total katiepress:         {len(kp_athletes):,}')
print(f'  Full names recovered:     {len(kp_athletes) - still_abbrev:,} ({(len(kp_athletes) - still_abbrev)/len(kp_athletes)*100:.1f}%)')
print(f'  Still abbreviated:        {still_abbrev}')

# Source-by-source samples
print(f'\n📦 SAMPLES BY SOURCE')
for src in ['keithgalli', 'paris2024', 'piterfm', 'katiepress']:
    sub = team_usa_athletes[team_usa_athletes['source'] == src]
    if len(sub) > 0:
        samples = sub.sample(min(3, len(sub)), random_state=42)
        names = ', '.join(samples['name'].tolist())
        print(f'  {src:15s} {names}')

print(f'\n{"=" * 70}')

In [ ]:
# ── Find and fix names with embedded newlines ────────────────────────

newline_names = team_usa_athletes[team_usa_athletes['name'].str.contains('\n', na=False)]
print(f'Names with embedded newlines: {len(newline_names)}\n')

for _, row in newline_names.iterrows():
    print(f'  athlete_id: {row["athlete_id"]}')
    print(f'  name: {repr(row["name"])}')
    print(f'  source: {row["source"]}')
    print()

# Also check profiles
newline_profiles = team_usa_athletes[team_usa_athletes['profile_summary'].str.contains('\n', na=False)]
print(f'Profiles with embedded newlines: {len(newline_profiles)}')

In [ ]:
# ── Phase 6.5, Step 4: Re-save cleaned data to GCS ──────────────────

ATHLETE_COLUMNS = [
    'athlete_id', 'name', 'gender', 'birth_date', 'birth_place', 'birth_country',
    'height_cm', 'weight_kg',
    'games_type', 'primary_sport', 'classification_code',
    'first_games_year', 'last_games_year', 'games_count',
    'gold_count', 'silver_count', 'bronze_count', 'total_medals',
    'reason', 'hero', 'philosophy', 'other_sports', 'coach',
    'hobbies', 'occupation', 'education',
    'profile_summary', 'embedding',
    'source',
]

RESULT_COLUMNS = [
    'athlete_id', 'athlete_name', 'games_year', 'games_season',
    'games_type', 'sport', 'discipline', 'event', 'medal',
    'classification_code',
]

# Drop temporary columns
if 'name_original' in team_usa_athletes.columns:
    team_usa_athletes.drop(columns=['name_original'], inplace=True)

# Strip newlines from all text fields (BQ doesn't handle quoted newlines by default)
text_cols = ['name', 'profile_summary', 'reason', 'hero', 'philosophy',
             'other_sports', 'coach', 'hobbies', 'occupation', 'education']
for col in text_cols:
    if col in team_usa_athletes.columns:
        count = team_usa_athletes[col].str.contains('\n', na=False).sum()
        if count > 0:
            print(f'  Stripped newlines from {col}: {count} rows')
            team_usa_athletes[col] = team_usa_athletes[col].str.replace('\n', ' ', regex=False)

# Sync athlete_name in results table
name_lookup = dict(zip(team_usa_athletes['athlete_id'], team_usa_athletes['name']))
team_usa_results['athlete_name'] = team_usa_results['athlete_id'].map(name_lookup).fillna(
    team_usa_results['athlete_name'])

# Apply column ordering
athletes_export = team_usa_athletes[[c for c in ATHLETE_COLUMNS if c in team_usa_athletes.columns]].copy()
results_export = team_usa_results[[c for c in RESULT_COLUMNS if c in team_usa_results.columns]].copy()

# Serialize embeddings as JSON
athletes_export['embedding'] = athletes_export['embedding'].apply(
    lambda x: json.dumps(x) if isinstance(x, list) else None)

ath_path = '/tmp/team_usa_athletes_final.csv'
res_path = '/tmp/team_usa_results_final.csv'

athletes_export.to_csv(ath_path, index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)
results_export.to_csv(res_path, index=False, encoding='utf-8')

!gcloud storage cp {ath_path} gs://class-demo/team-usa/final/team_usa_athletes.csv
!gcloud storage cp {res_path} gs://class-demo/team-usa/final/team_usa_results.csv

!gcloud storage ls -l gs://class-demo/team-usa/final/

print(f'\n✅ Cleaned data saved to gs://class-demo/team-usa/final/')
print(f'   Athletes: {len(athletes_export):,} rows')
print(f'   Results:  {len(results_export):,} rows')